# Programming for Data Analysis Assignment 2

Author - Sean Humphreys

## Contents

1. [Problem Statement](#problem-statement)

1. [Software Libraries](#software-libraries)

2. [Data Exploration and Cleansing](#data-exploration-and-cleansing)

    1. [Carbon Dioxide Data](#co2-data)

        1. [Luthi et al. co2 Data](#luthi-et-al-co2-data)

        2. [IPCC CO2 Data](#ipcc-CO2-data)

        3. [Mauna Loa CO2 Data](#mauna-loa-co2-data)

    2. [Temperature Data](#temperature-data)

        1. [Jouzel (2007) Temperature Data](#jouzel-2007-temperature-data)

        2. [NOAA Temperature Data](#noaa-temperature-data)

        3. [Further Temperature Data](#further-temperature-data)

        4. [Monthly Temperature Data](#monthly-temperature-data)

    3. [Methane Data](#methane-data)

    4. [Irish Data](#irish-data)

        1. [Irish Temerature Data](#irish-temperature-data)

        2. [Irish Precipitation Data](#irish-precipitation-data)

    4. [Fused Dataset](#fused-dataset)


2. [Analysis](#analysis)

    1. [Carbon Dioxide](#carbon-dioxide)

    2. [Temperature](#temperature)

    3. [Methane](#methane)

3. [Irish Context](#irish-context)

    1. [Irish Context - Temperature](#irish-context---temperature)
    
    2. [Irish Context - Precipitation](#irish-context---precipitation)

4. [Predictive Model](#predictive-model)

2. [References](#references)

3. [Associated Reading](#associated-reading)

---

## Problem Statement <a id="problem-statement"></a>

+ Analyse CO2 vs Temperature Anomaly from 800kyrs – present.

+ Examine one other (paleo/modern) features (e.g. CH4 or polar ice-coverage)

+ Examine Irish context:
    
    + [Climate change signals](/literature/the_emergence_of_a_climate_change_signal_in_long_term_irish_meteorological_observations.pdf) : (see Maynooth study: The emergence of a climate change signal in long-term Irish meteorological observations - ScienceDirect)

+ Fuse and analyse data from various data sources and format fused data set as a pandas dataframe and export to csv and json formats

+ For all of the above variables, analyse the data, the trends and the relationships between them (temporal leads/lags/frequency analysis).

+ Predict global temperature anomaly over next few decades (synthesise data) and compare to published climate models if atmospheric CO2 trends continue

+ Comment on accelerated warming based on very latest features (e.g. temperature/polar-ice-coverage)

---

## Software Libraries <a id="software-libraries"></a>

- [Datetime](https://docs.python.org/3/library/datetime.html) (https://docs.python.org/3/library/datetime.html - last accessed 21 Dec. 2023) - supplies classes for manipulating dates and times.

- [Matplotlib](https://matplotlib.org/) (https://matplotlib.org/ - last accessed 13 Dec. 2023) - is an open-source software library for creating static, animated, and interactive visualisations in Python.

- [Numpy](https://numpy.org/doc/stable/index.html) (https://numpy.org/doc/stable/index.html - last accessed 8. Nov 2023) - is an open-source software library for the Python, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.

- [Pandas](https://pandas.pydata.org/) (https://pandas.pydata.org/ - last accessed 3 Nov. 2023) is an open-source software library used in data analytics that allows data analysis and manipulation. Pandas is built on top of the Python programming language. A Pandas DataFrame is a dictionary like container for series objects. A DataFrame is the primary Pandas data structure.

- [scikit-learn](https://scikit-learn.org/stable/index.html) (https://scikit-learn.org/stable/index.html) - are a set of tools used for predictive analysis. It is used extensively in machine learning.

- [SciPy](https://scipy.org/) (https://scipy.org/ - last accessed 21 Nov. 2023) provides algorithms for scientific computing in Python. SciPy is a portmanteau of Scientific Python. It is a scientific computation library that uses Numpy underneath.

- [Seaborn](https://seaborn.pydata.org/) (https://seaborn.pydata.org/ - last accessed 23 Nov. 2023) - is visualisation software that is built on matplotlib. It offers a high-level interface for drawing attractive and informative statistical graphics.

In [2]:
# import the required software libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import signal
from datetime import date

---

## Data Exploration and Cleansing <a id="data-exploration-and-cleansing"></a>

The Pandas software library is used to clean and process datasets. 

### Carbon Dioxide Data <a id="carbon-dioxide-data"></a>

####  Luthi et al. Carbon Dixoide Data <a id="luthi-et-al.-carbon-dioxide-data"></a>

The first carbon dataset (CO2) is sourced from Luthi et al (2008.) This dataset is a composite of a number of samples of CO2 data from studies on antarctic ice core analysis. Luthi et al. use the EDC3 chronology to describe the age of the samples in the dataset. This EDC3 chronology discussed further [here](#data-analysis). 

Historic CO2 data in this dataset ranges from the year 1813 to 800k years before present.

To process the data it is read in as a Pandas DataFrame from separate worksheets in an Microsoft Excel file.

In [4]:
# code adapted from https://www.mybib.com/tools/harvard-referencing-generator [Accessed 19 Dec. 2023]
co2_ipcc = pd.read_excel(
    'datasets/historic/co2/41586_2008_BFnature06949_MOESM31_ESM.xls', sheet_name='2.  Vostok-TD-Dome C')
co2_ipcc_new = pd.read_excel(
    'datasets/historic/co2/41586_2008_BFnature06949_MOESM31_ESM.xls', sheet_name='1.  new CO2 data')

DataFrames to capture specific subsets of data from assorted studies in the master dataset are defined. These subsets will be stitched together to create a composite dataset of historic co2 data.

In [5]:
# adapted from code found here - https://www.shanelynn.ie/pandas-iloc-loc-select-rows-and-columns-dataframe/ [Accessed 19 Dec. 2023].
monnin_luthi = co2_ipcc.iloc[6:189, 1:3]
pettit_luthi = co2_ipcc.iloc[19:353, 5:7]
siegenthaler_1_luthi = co2_ipcc.iloc[6:26, 16:18]
siegenthaler_2_luthi = co2_ipcc.iloc[6:328, 12:14]
luthi_luthi = co2_ipcc_new.iloc[16:253, 1:3]

The columns in each dataframe are renamed to logical names.

In [6]:
monnin_luthi.rename(columns=({'Unnamed: 1':'yr_bp', 'Unnamed: 2':'co2_ppmv'}), inplace=True)
pettit_luthi.rename(columns=({'Unnamed: 5':'yr_bp', 'Unnamed: 6':'co2_ppmv'}), inplace=True)
siegenthaler_1_luthi.rename(columns=({'Unnamed: 16':'yr_bp', 'Unnamed: 17':'co2_ppmv'}), inplace=True)
siegenthaler_2_luthi.rename(columns=({'Unnamed: 12':'yr_bp', 'Unnamed: 13':'co2_ppmv'}), inplace=True)
luthi_luthi.rename(columns=({'Unnamed: 1':'yr_bp', 'Unnamed: 2':'co2_ppmv'}), inplace=True)

A function is defined to carry out a number of processing actions on each DataFrame. The `year()` function:

+ creates a columns that calculates the NUMBER OF yearS before 2023

+ creates a column that calculates the year

+ drops any rows with null values

In [7]:
def year(sample):
    sample['years_before_2023'] = 2023 - (1950-(sample['yr_bp']))
    sample['year'] = 1950 - (sample['yr_bp'])
    sample.drop('yr_bp', axis=1, inplace=True)
    sample.dropna(axis=0, inplace=True)
    return sample

Using a for loop, each of the subsets of data can be passed to the `year()` function.

In [8]:
studies = [monnin_luthi ,pettit_luthi, siegenthaler_1_luthi, siegenthaler_2_luthi, luthi_luthi]

for study in studies: year(study)

Each of the subsets of CO2 data is concatenated into one DataFrame to create a composite of the data in Luthi et al (2008) so that the trending carbon dioxide levels for the 800,000 year period can be plotted.

In [9]:
# code adapted from https://pandas.pydata.org/docs/reference/api/pandas.concat.html [Accessed 13 Dec. 2023]. 
luthi_frames = [monnin_luthi ,pettit_luthi, siegenthaler_1_luthi, siegenthaler_2_luthi, luthi_luthi]

luthi_full_co2_data = pd.concat(luthi_frames, ignore_index = True)

The concatenated dataset is explored.

In [10]:
luthi_full_co2_data.head()

,co2_ppmv,years_before_2023,year
0,280.4,210,1813
1,274.9,341,1682
2,277.9,352,1671
3,279.1,468,1555
4,281.9,477,1546


In [11]:
luthi_full_co2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   co2_ppmv           1096 non-null   object
 1   years_before_2023  1096 non-null   object
 2   year               1096 non-null   object
dtypes: object(3)
memory usage: 25.8+ KB


Variables are cast to Pandas Dtypes that can be used for analysis.

In [12]:
# code adapted from https://www.geeksforgeeks.org/python-pandas-dataframe-astype/ [Accessed 15 Dec. 2023].
luthi_full_co2_data = luthi_full_co2_data.astype({'co2_ppmv':'float64', 'years_before_2023': 'int', 'year':'int'})

In [13]:
luthi_full_co2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   co2_ppmv           1096 non-null   float64
 1   years_before_2023  1096 non-null   int32  
 2   year               1096 non-null   int32  
dtypes: float64(1), int32(2)
memory usage: 17.3 KB


The DataFrame is insepcted for Duplicate vales in the *years_before_2023* column.

In [14]:
# code adapted from https://stackoverflow.com/questions/14657241/how-do-i-get-a-list-of-all-the-duplicate-items-using-pandas-in-python Accessed 22 Dec. 2023
pd.concat(g for _, g in luthi_full_co2_data.groupby("years_before_2023") if len(g) > 1)

,co2_ppmv,years_before_2023,year
530,275.2,409456,-407433
531,274.2,409456,-407433


As can be seen there is two entries for the year -407433. The simplest way to manage this is to drop the year with the smaller value. As the difference is 1 ppmv the effect is negligible.

In [15]:
luthi_full_co2_data.drop(531, axis=0, inplace=True)

The columns are reordered to a more logical arrangement.

In [16]:
# adapted from code found here:
# https://practicaldatascience.co.uk/data-science/how-to-reorder-pandas-dataframe-columns [Accessed 13 Dec. 2023]
luthi_full_co2_data = luthi_full_co2_data.reindex(columns=['years_before_2023', 'year', 'co2_ppmv'])

In [17]:
luthi_full_co2_data.head()

,years_before_2023,year,co2_ppmv
0,210,1813,280.4
1,341,1682,274.9
2,352,1671,277.9
3,468,1555,279.1
4,477,1546,281.9


---

####  IPCC CO2 Data <a id="ipcc-co2-data"></a>

A second CO2 dataset is sourced from the [The Intergovernmental Panel on Climate Change (IPCC)](https://www.ipcc.ch/) (https://www.ipcc.ch/). This dataset is a composite of a number of samples of carbon dioxide data from studies on ice core analysis. The samples have been dated using the The Antarctic Ice Core Chronology (AICC2012) system. This is discussed further [here](#data-analysis).

This dataset contains measurements of CO2. The range of years in the composite dataset is from 1950 to 803,182 years from 2023.

The excel spreadsheet with historic IPCC co2 data is read in as a Pandas Dataframe.

In [18]:
ipcc_c02_data = pd.read_excel(
                              'datasets/historic/co2/grl52461-sup-0003-supplementary.xls', 
                              sheet_name='all records')

DataFrames to capture specific subsets of data from assorted studies in the master dataset are defined. These subsets will be stitched together to create a composite dataset of historic co2 data.

In [19]:
# use iloc to create subsets of data from the master dataset
rubino = ipcc_c02_data.iloc[90:, [83, 86]]
macfarling = ipcc_c02_data.iloc[137:, 68:70]
monnin = ipcc_c02_data.iloc[25:120, 2:4]
marcott = ipcc_c02_data.iloc[31:321, 98:100]
ahn = ipcc_c02_data.iloc[7:202, 89:91]
bereiter = ipcc_c02_data.iloc[28:106, 34:36]
bereiter_2 = ipcc_c02_data.iloc[60:154, 39:41]
schneider = ipcc_c02_data.iloc[6:, 65:67]
petit = ipcc_c02_data.iloc[124:348, 7:9]
siegenthaler = ipcc_c02_data.iloc[6:26, 20:22]
siegenthaler_2 = ipcc_c02_data.iloc[6:226, 15:17]
bereiter_3 = ipcc_c02_data.iloc[37:, 102:104]

The columns in each dataframe are renamed to logical names

In [20]:
rubino.rename(columns=({'Unnamed: 83':'yr_bp', 'Unnamed: 86':'co2_ppmv'}), inplace=True)
macfarling.rename(columns=({'Law Dome (0-2 kyr BP)':'yr_bp', 'Unnamed: 69':'co2_ppmv'}), inplace=True)
monnin.rename(columns=({'Unnamed: 2':'yr_bp', 'Unnamed: 3':'co2_ppmv'}), inplace=True)
marcott.rename(columns=({'Unnamed: 98':'yr_bp', 'Unnamed: 99':'co2_ppmv'}), inplace=True)
ahn.rename(columns=({'Unnamed: 89':'yr_bp', 'Unnamed: 90':'co2_ppmv'}), inplace=True)
bereiter.rename(columns=({'Unnamed: 34':'yr_bp', 'Unnamed: 35':'co2_ppmv'}), inplace=True)
bereiter_2.rename(columns=({'Unnamed: 39':'yr_bp', 'Unnamed: 40':'co2_ppmv'}), inplace=True)
schneider.rename(columns=({'Unnamed: 65':'yr_bp', 'Unnamed: 66':'co2_ppmv'}), inplace=True)
petit.rename(columns=({'Unnamed: 7':'yr_bp', 'Unnamed: 8':'co2_ppmv'}), inplace=True)
siegenthaler.rename(columns=({'Unnamed: 20':'yr_bp', 'Unnamed: 21':'co2_ppmv'}), inplace=True)
siegenthaler_2.rename(columns=({'Unnamed: 15':'yr_bp', 'Unnamed: 16':'co2_ppmv'}), inplace=True)
bereiter_3.rename(columns=({'Unnamed: 102':'yr_bp', 'Unnamed: 103':'co2_ppmv'}), inplace=True)


Using a *for loop*, a number of actions are performed on each of the subsets of data. In the source data the year is a floating point integer. That will cause an inaccuracy of 1 year when calculating the number of years before 2023. To account for this the floating point integer in the *yr_bp* column is rounded down to the nearest whole number.

In [21]:
studies = [rubino, macfarling, monnin, marcott, ahn, bereiter, bereiter_2, 
           schneider, petit, siegenthaler, siegenthaler_2, bereiter_3]

# the last line in the function is adapted from code found here:
# https://stackoverflow.com/questions/35873927/rounding-down-values-in-pandas-dataframe-column-with-nans
# accessed 23 Dec. 2023
for study in studies:
    study.dropna(axis=0, inplace=True)
    study['yr_bp'].astype(int)
    study['yr_bp'] = study['yr_bp'].apply(np.floor)

Each of the elements in the *studies* list is passed to the `year()` function.

In [22]:
for study in studies: year(study)

Each of the subsets of CO2 data is concatenated into one DataFrame to create a composite of the subsets of data. FRom this DataFrame the trending levels of carbon dioxide over time are plotted.

In [23]:
# code adapted from https://pandas.pydata.org/docs/reference/api/pandas.concat.html [Accessed 13 Dec. 2023].
frames = [rubino, macfarling, monnin, marcott, ahn, bereiter, bereiter_2, schneider, petit, siegenthaler, siegenthaler_2, bereiter_3]

ipcc_full_co2_data = pd.concat(frames, ignore_index = True)

The `info ()` gives an indication of the datatypes in the concatenated DataFrame. All variables are *objects*. This limits they type of analysis that can be carried out on these variables.

In [24]:
ipcc_full_co2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852 entries, 0 to 1851
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   co2_ppmv           1852 non-null   object
 1   years_before_2023  1852 non-null   object
 2   year               1852 non-null   object
dtypes: object(3)
memory usage: 43.5+ KB


The Pandas `astype()` is used to cast a Pandas object to a specific type of datatype.

In [25]:
ipcc_full_co2_data = ipcc_full_co2_data.astype({'co2_ppmv':'float64', 'years_before_2023':'int', 'year':'int'})

In [26]:
ipcc_full_co2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852 entries, 0 to 1851
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   co2_ppmv           1852 non-null   float64
 1   years_before_2023  1852 non-null   int32  
 2   year               1852 non-null   int32  
dtypes: float64(1), int32(2)
memory usage: 29.1 KB


Upon checking the first five rows of the DataFrame it is apparent that some years have more than one entry. These are not duplicate rows as such but rather years where there more than one measurement of CO2 was taken.

In [27]:
ipcc_full_co2_data.head()

,co2_ppmv,years_before_2023,year
0,316.334020,64,1959
1,316.101403,64,1959
2,314.572166,65,1958
3,315.270355,65,1958
4,316.325380,65,1958


Checking for duplicate values in the *years_before_2023* column reveals that there is 62 rows that have the same value for the *year_before_2023* column. The approach to deal with these years is to drop the duplicates but keep the first row. This is blunt but due to the minimal difference between the CO2 value for each duplicate, the effect on any analysis carried out on the dataset will be negligible.

In [28]:
pd.concat(g for _, g in ipcc_full_co2_data.groupby("years_before_2023") if len(g) > 1)

,co2_ppmv,years_before_2023,year
0,316.334020,64,1959
1,316.101403,64,1959
2,314.572166,65,1958
3,315.270355,65,1958
4,316.325380,65,1958
...,...,...,...
106,284.001634,172,1851
109,283.182872,178,1845
110,283.531124,178,1845
1264,275.200000,409671,-407648


Duplicates are removed from the *years_before_2023* column.

In [29]:
ipcc_full_co2_data.drop_duplicates(subset='years_before_2023', keep='first', inplace=True)

Checking the first five rows of the DataFrame confirms that the duplicate entries have been dropped from the dataset.

In [30]:
ipcc_full_co2_data.head()

,co2_ppmv,years_before_2023,year
0,316.334020,64,1959
2,314.572166,65,1958
6,315.342074,66,1957
7,314.713961,68,1955
8,313.171057,69,1954


The columns are rearranged to be consistent with the *luthi_full_co2_data* DataFrame.

In [31]:
ipcc_full_co2_data = ipcc_full_co2_data.reindex(columns=['years_before_2023', 'year', 'co2_ppmv'])

In [32]:
ipcc_full_co2_data.head()

,years_before_2023,year,co2_ppmv
0,64,1959,316.334020
2,65,1958,314.572166
6,66,1957,315.342074
7,68,1955,314.713961
8,69,1954,313.171057


---

####  Mauna Loa CO2 Data <a id="mauna-loa-cO2-data"></a>

The most recent CO2 dataset in a Comma Separated Value (CSV) file is available from from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.csv. This dataset contains measurements of annual mean CO2 expressed as a mole fraction in dry air. The range of years in the dataset is from 1959 to 2022.

Using Pandas the CSV file is read in as a Pandas DataFrame.

In [33]:
# https://gml.noaa.gov/ccgg/trends/data.html
mauna_loa_co2 = pd.read_csv('https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.csv', skiprows=43)

The columns in the dataset are renamed to logical names.

In [34]:
# code adapted from # https://sparkbyexamples.com/pandas/rename-columns-with-list-in-pandas-dataframe/
cols = ['year', 'co2_ppmv', 'unc']

mauna_loa_co2.columns = cols

An unnecessary column is removed from the dataset.

In [35]:
# code adapted from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html [Accessed 13 Dec. 2023]
mauna_loa_co2.drop(['unc'], axis=1, inplace=True)

An additional column is added to the dataset that calculates the *year before the 2023*.

In [36]:
mauna_loa_co2['years_before_2023'] = 2023 - mauna_loa_co2['year']

# sort the data based on the year before present. Based on code from - https://saturncloud.io/blog/how-to-sort-pandas-dataframe-from-one-column/ [Accessed 13 Dec. 2023].
mauna_loa_co2 = mauna_loa_co2.sort_values('years_before_2023')

The columns in the dataset are reordered and the first five rows of the DataFrame are checked.

In [37]:
# adapted from code found here - https://practicaldatascience.co.uk/data-science/how-to-reorder-pandas-dataframe-columns [Accessed 13 Dec. 2023]
mauna_loa_co2 = mauna_loa_co2.reindex(columns=['co2_ppmv', 'year', 'years_before_2023'])

In [38]:
mauna_loa_co2.head()

,co2_ppmv,year,years_before_2023
63,418.53,2022,1
62,416.41,2021,2
61,414.21,2020,3
60,411.65,2019,4
59,408.72,2018,5


The DataFrame Dtypes are checked and are found to be consistent with the DataFrames containing CO2 data previously created.

In [39]:
mauna_loa_co2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 63 to 0
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   co2_ppmv           64 non-null     float64
 1   year               64 non-null     int64  
 2   years_before_2023  64 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 2.0 KB


Any rows with duplicate years are dropped.

In [40]:
mauna_loa_co2.drop_duplicates(subset='years_before_2023', keep='first', inplace=True)

The order of the columns is changes to be consistent with the other DataFrames containing CO2 data.

In [41]:
mauna_loa_co2 = mauna_loa_co2.reindex(columns=['years_before_2023', 'year', 'co2_ppmv'])

In [42]:
mauna_loa_co2.head()

,years_before_2023,year,co2_ppmv
63,1,2022,418.53
62,2,2021,416.41
61,3,2020,414.21
60,4,2019,411.65
59,5,2018,408.72


####  Global Monthly CO2 Data 1979 - 2023 <a id="global-monthly-c02-data-1979---2023"></a>

Global monthly CO2 data from the NOAA is available from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.txt (Accessed 23 Dec. 2023).

This data is read in as a Pandas DataFrame

In [43]:
monthly_co2_data = pd.read_csv('datasets/historic/co2/co2_mm_gl.csv', skiprows=38)

The first 5 rows of the dataset are inspected.

In [44]:
monthly_co2_data.head()

,year,month,decimal,average,average_unc,trend,trend_unc
0,1979,1,1979.042,336.56,0.11,335.92,0.09
1,1979,2,1979.125,337.29,0.09,336.25,0.10
2,1979,3,1979.208,337.88,0.11,336.51,0.10
3,1979,4,1979.292,338.32,0.12,336.72,0.10
4,1979,5,1979.375,338.26,0.03,336.71,0.10
...,...,...,...,...,...,...,...
532,2023,5,2023.375,420.54,0.10,418.87,0.06
533,2023,6,2023.458,419.53,0.10,418.94,0.06
534,2023,7,2023.542,417.83,0.10,419.10,0.06
535,2023,8,2023.625,416.53,0.10,419.36,0.06


Columns that will not be used are dropped from the DataFrame.

In [45]:
monthly_co2_data.drop(['decimal', 'average_unc', 'trend', 'trend_unc'], axis=1, inplace=True)

The variable Dtypes are checked.

In [46]:
monthly_co2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     537 non-null    int64  
 1   month    537 non-null    int64  
 2   average  537 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 12.7 KB


The values in the variables in the *year* and *month* columns are combined and appended to an empty list.  A *for loop* is used to iterate through every row in the DataFrame to complete this operation for every row in the DataFrame.

 The list is subsequently assigned to the DataFrame as the *date* column.

In [47]:
# adapted from code found at 
# - https://stackoverflow.com/questions/48304927/cleanly-combine-year-and-month-columns-to-single-date-column-with-pandas 
# [Accessed 21 Dec. 2023].

DATE = []
for y, m in zip(monthly_co2_data.year, monthly_co2_data.month):
    DATE.append(date(y, m, 1))
    
monthly_co2_data['date'] = DATE

The first five rows of the DataFrame are inspected.

In [48]:
monthly_co2_data

,year,month,average,date
0,1979,1,336.56,1979-01-01
1,1979,2,337.29,1979-02-01
2,1979,3,337.88,1979-03-01
3,1979,4,338.32,1979-04-01
4,1979,5,338.26,1979-05-01


The *date* column is set as the DataFrame index.

In [49]:
monthly_co2_data = monthly_co2_data.set_index('date').asfreq('MS')

This operation is validated by visually checking the first five rows of the DataFrame.

In [51]:
monthly_co2_data

,year,month,average
date,,,
1979-01-01,1979,1,336.56
1979-02-01,1979,2,337.29
1979-03-01,1979,3,337.88
1979-04-01,1979,4,338.32
1979-05-01,1979,5,338.26
...,...,...,...
2023-05-01,2023,5,420.54
2023-06-01,2023,6,419.53
2023-07-01,2023,7,417.83


---

### Temperature Data <a id="temperature-data"></a>

#### Jouzel (2007) Temperature Data <a id="jouzel-(2007)-temperature-data"></a>

Data from Jouzel et al (2007) is from a high-resolution deuterium profile from European Project for Ice Coring in Antarctica (EPICA) Dome C ice core. As per the metadata available for this data set, available from [here](https://doi.pangaea.de/10.1594/PANGAEA.683655) (https://doi.pangaea.de/10.1594/PANGAEA.683655), temperature was estimated after correction for sea-water isotopic composition (Bintanja, van de Wal and Oerlemans, 2005) and for ice sheet elevation on EDC3 age scale (Parrenin et al, 2007). The EDC3 age scale is discussed further [here](#carbon-dioxide).

The dataset is read into a Pandas DataFrame using Python.

In [ ]:
jouzel_temp = pd.read_csv('epicaDC.deuttemp.EDC3-AICC.csv')

The *jouzel_temp* DataFrame is explored to understand the data in the dataset. The first and last 5 rows of the DatFrame are printed. From visualising this information the dataset appears to consist of seven variables. There appears to be a *temp* variable that is in Kelvin. Two variables that describe a chronology for the temperature value - *EDC3* and *AICC2012*. The systems are discussed in further detail [here](#carbon-dioxide).

From the data in this dataset it should be possible to calculate the year when the corresponding temperature measurement was taken. Once this is known a temperature anomaly value can be calculated.

Temperature anomaly is defined as:

*"The term temperature anomaly means a departure from a reference value or long-term average. A positive anomaly indicates that the observed temperature was warmer than the reference value, while a negative anomaly indicates that the observed temperature was cooler than the reference value."* (www.ncei.noaa.gov, n.d.).

From the Jouzel dataset the following measurements can be calculated:

+ age of a sample

+ temperature value in degrees celsius of the sample

+ temperature anomaly for a desired period of the sample

In [ ]:
# print the first five rows of data
jouzel_temp.head()

In [ ]:
jouzel_temp.tail()

The Pandas data types can be understood by using the `info()` method.

In [ ]:
jouzel_temp.info()

The NOAA temperature data explored in the next [section](#noaa-temperature-data) is a subset of this data using the EDC3 chronology with a temperature anomaly calculated as the temperature difference from the average of the last 1000 years.

It should therefore be possible to create a subset of this data for the AICC2012 chronology with a temperature anomaly  calculated as the temperature difference from the average of the last 1000 years.

Firstly, unnecessary columns are dropped from DataFrame.

In [ ]:
jouzel_temp.drop(['bag', 'ztop', 'deutfinal', 'acc-EDC3beta'], axis = 1, inplace = True)

A column to convert Kelvin to celsius is added.

In [ ]:
jouzel_temp['celsius'] = jouzel_temp['temp'] - 273.15

The mean temperature for approximately the last 1000 years is assigned to a variable

In [ ]:
average = jouzel_temp['celsius'][0:87].mean()

The temperature anomaly in celsius is calculated.

In [ ]:
jouzel_temp['anomaly'] = jouzel_temp['celsius'] - average

A column to calculate the years from present is added to the DataFrame.

In [ ]:
# calculate years before present aicc2012 chronology
jouzel_temp['aicc2012_years_before_2023'] = 2023 - (1950 - jouzel_temp['AICC2012'])

# calculate years before present EDC3 chronology
jouzel_temp['edc3_years_before_2023'] = 2023 - (1950 - jouzel_temp['EDC3béta'])

The DataFrame is printed to validate the calculations.

In [ ]:
jouzel_temp.head()

The DataFrame Pandas Dtypes are explored.

In [ ]:
jouzel_temp.info()

The *years_before_2023* variables are converted to integers.

In [ ]:
jouzel_temp = jouzel_temp.astype({'aicc2012_years_before_2023':'int', 'edc3_years_before_2023':'int'})

In [ ]:
jouzel_temp.info()

In [ ]:
jouzel_temp.head()

Any rows with duplicate years are dropped from the dataset.

In [ ]:
jouzel_temp.drop_duplicates(subset='aicc2012_years_before_2023', keep='first', inplace=True)

In [ ]:
jouzel_temp.head()

#### NOAA Temperature Data <a id="noaa-temperature-data"></a>

The NOAA is the National Oceanic and Atmospheric Administration. More information on this organisation is available from [here](https://www.noaa.gov/) (https://www.noaa.gov/).

Upon exploration of the NOAA temperature dataset it is apparent the temperature data from the NOAA is essentially a subset of data from Jouzel et al (2007) that has been processed to calculate degrees celsius temperature anomaly. As per the metadeta for the sample, availalbe [here](https://doi.pangaea.de/10.1594/PANGAEA.683655) (https://doi.pangaea.de/10.1594/PANGAEA.683655), the core samples in the dataset have been aged using the EDC3 chronology, discussed further [here](#analysis).

As per the metadata for this dataset the temperature anomaly is calculated as the temperature difference from the average of the last 1000 years.

The data in this is from the year 2000 AD to approximately 800,000 years ago.

Temperature data from the NOAA is read in as a Pandas DataFrame.

In [ ]:
# https://stackoverflow.com/questions/27896214/reading-tab-delimited-file-with-pandas-works-on-windows-but-not-on-mac

noaa_temp = pd.read_csv('https://doi.pangaea.de/10.1594/PANGAEA.683655?format=textfile', sep="\t", skiprows=20)

In [ ]:
noaa_temp.head()

In [ ]:
noaa_temp.tail()

The columns are renamed to logical names.

In [ ]:
noaa_temp.rename(columns=({"Depth ice/snow [m]":"depth_ice_snow_m", "Age model [ka]":"age_model_ka", "δD [‰ SMOW]":"δd_‰_smow", "delta T [°C]":"delta_t_c", "Sample ID":"sample_id"}), inplace=True)

In [ ]:
noaa_temp.head()

Columns are added to calculate the year and the year before 2023.

In [ ]:
noaa_temp['years_before_2023'] = 2023 - (1950-(noaa_temp['age_model_ka']*1000))

In [ ]:
noaa_temp.head()

The Dtypes in the *noaa_temp* DataFrame are explored.

In [ ]:
noaa_temp.info()

The *years_before_2023* variable is cast from a float to an integer

In [ ]:
noaa_temp = noaa_temp.astype({'years_before_2023':'int'})

In [ ]:
noaa_temp.info()

To understand if the transform that was applied to the Jouzel temperature dataset matches the above dataset they are plotted against one another.

As can be seen the manually transformed dataset does not cancel out the data from the NOAA. For the first 400,000 they are very closely matched. After this the temperature anonalies differ as does the time scales which suggests that before 400,000 years ago the data is somehow processed differntly by the NOAA.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 10 ))

# plot the results
ax.plot(jouzel_temp['edc3_years_before_2023'], jouzel_temp['anomaly'], label = 'Manually Transformed EDC3 Temp Data')

# adapted from code found here - https://www.geeksforgeeks.org/change-the-line-opacity-in-matplotlib/ [Accessed 19 Dec. 2023]
ax.plot(noaa_temp['years_before_2023'], noaa_temp['delta_t_c'], label = 'Anomaly as per NOAA EDC3 Chronology', color = 'black', alpha = 0.6)
ax.set_xlabel('Years Before 2023')
ax.set_ylabel('Temperature Anomaly Degrees Celsius')

ax.legend()
ax.set_title('EDC 2023 Manual Anomaly Calculation Vs NOAA Calculation', weight ='bold', size=15)
plt.show()

#### Further Temperature Data <a id="further-temperature-data"></a>

A further set of historic temperature data can be created from numerous other data sources. 

The temperature from from 1880 to 2022 in this dataset is sourced from [NASA](https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt) (https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt [Accessed 12 Dec. 2023].). [1] The temperature data is the global surface temperature.

Temperature data from 1880 to 800k years from the 2023 was sourced from [https://www.temperaturerecord.org/#sources](https://www.temperaturerecord.org/#sources) accessed 13 Dec. 2023. [2] & [3]. 

Data ranging rom 2000 to 20,000 years ago reconstructed by averaging well-dated, calibrated proxy temperature records from around the world, mostly from ocean margin sediment cores, in addition to lake and ice cores on land.

Data from 20,000 to 800,000 years ago is reconstructed by taking a spatially-weighted average of 59 proxy sea surface temperature records from around the global oceans.

Data from 2000 to 800,000 years ago has been scaled to have a glacial-interglacial range of 4°C based on more comprehensive datasets suggesting that this was the likely magnitude of ice age cooling (Annan and Hargreaves, 2013, Climate of the Past). 

All of the temperature data is compared to the long-term average from 1951 to 1980. Therefore this temperature data uses a different anomaly to the NOAA data and should have different plots when compared to each other.

[1] Credits - Snyder, C.W. 2016.

[2] Credits - Marcott et al, 2013

[3] Credits - Shakun et al, 2012



Using Pandas modern temperature data is read in from the NASA website.

In [ ]:
nasa_temp = pd.read_csv(
    'https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt', 
    skiprows=5, header=None, sep = ' ', skipinitialspace=True, engine='python', names=['year', 'temp_anomaly', 'lowness'])

An unnecessary column is dropped from the DataFrame.

In [ ]:
nasa_temp.drop(['lowness'], axis=1, inplace=True)

A column with the year before present is created.

In [ ]:
nasa_temp['years_before_2023'] = 2023 - nasa_temp['year']

The remaining columns are renamed to a standard naming convention that will be used with temperature data from another source.

In [ ]:
nasa_temp = nasa_temp.reindex(columns=['year', 'years_before_2023', 'temp_anomaly'])

The NASA data is sorted by year.

In [ ]:
nasa_temp = nasa_temp.sort_values('year', ascending=False)

In [ ]:
nasa_temp

Pre 1800's temperature data is read in from worksheets in an excel spreadsheet that contains all of the historic temperature data.

In [ ]:
moberg_temp = pd.read_excel('temperature_dataset.xlsx', 
                            sheet_name='2,000 yr',  names=['year', 'yr_bp', 'temp_anomaly', 'x', 'y', 'z'])
clark_temp = pd.read_excel('temperature_dataset.xlsx', 
                           sheet_name='20,000 yr', names=['yr_bp', 'temp_anomaly', 'x', 'y', 'z'])
shakun_temp = pd.read_excel('temperature_dataset.xlsx', 
                            sheet_name='800,000 yr', names=['yr_bp', 'temp_anomaly', 'x', 'y', 'z'])


Unneeded columns are removed from the DataFrames.

The remaining columns are re-ordered to make them consistent with the rest of the temperature DataFrames.

In [ ]:
moberg_temp.drop(moberg_temp.index[0:100], axis = 0, inplace = True)

The unneeded *year* column is dropped. 

In [ ]:
moberg_temp.drop('year', axis=1, inplace=True)

A number of precessing tasks are grouped together in a function. The `temp_year()` function removes unneeded columns from the DataFrame and adds a column to calculate the year.

In [ ]:
def temp_year(sample):
    sample.drop(['x', 'y', 'z'],axis=1, inplace=True)
    sample['year'] = 1950-sample['yr_bp']
    sample['years_before_2023'] = 2023 - sample['year']
    sample.drop(['yr_bp'],axis=1, inplace=True)
    return sample

Using a for loop the relevant datasets are paased to the `temp_year()` function.

In [ ]:
samples = [moberg_temp, clark_temp, shakun_temp]

for sample in samples: temp_year(sample)

Rows are dropped from each of the  DataFrames so that there is no overlap between them.

In [ ]:
clark_temp.drop(clark_temp.index[0:19], axis = 0, inplace = True)
shakun_temp.drop(shakun_temp.index[0:7], axis = 0, inplace = True)

In [ ]:
moberg_temp = moberg_temp.reindex(columns=['year', 'years_before_2023', 'temp_anomaly'])
clark_temp = clark_temp.reindex(columns=['year', 'years_before_2023', 'temp_anomaly'])
shakun_temp = shakun_temp.reindex(columns=['year', 'years_before_2023', 'temp_anomaly'])

All of the temperature DataFrames are concatenated to give a composite record of the temperature anomaly over the last 800k years.

In [ ]:
frames_temp = [nasa_temp, moberg_temp, clark_temp, shakun_temp]

full_temp_data = pd.concat(frames_temp, ignore_index = True)

The variable Dtypes are checked in the *full_temp_data*

In [ ]:
full_temp_data.info()

Duplicate year entries are removed from the DataFrame.

In [ ]:
full_temp_data.drop_duplicates(subset='years_before_2023', keep='first', inplace=True)

In [ ]:
full_temp_data

#### Monthly Temperature Data <a id="monthly-temperature-data"></a>

Monthly global temperature anomaly temperature data was sourced from https://datahub.io/core/global-temp#readme (Accessed 28 Dec. 2023). The source of the data focused on is GISTEMP: NASA Goddard Institute for Space Studies (GISS) Surface Temperature Analysis, Global Land-Ocean Temperature Index.

Read in the dataset as a Pandas DataFrame.

In [ ]:
monthly_temperature_data = pd.read_csv('https://datahub.io/core/global-temp/r/monthly.csv')

Filter to show only data from GISTEMP.

In [ ]:

monthly_temperature_data = monthly_temperature_data[monthly_temperature_data['Source'] == 'GISTEMP']

In [ ]:
monthly_temperature_data

In [ ]:
monthly_temperature_data.sort_index(ascending=False, inplace=True)

monthly_temperature_data.reset_index(drop=True)

Drop *Source* column as it is not needed.

Insoect the DataFrame variables.

In [ ]:
monthly_temperature_data.info()

Covert the *Date* variable to a Pandas *datetime* Dtype.

In [ ]:
dates = pd.date_range(start='12/06/1879', end='12/12/2016', freq='MS')

In [ ]:
monthly_temp_data = pd.DataFrame({'mean':monthly_temperature_data['Mean'].values}, index=dates)

In [ ]:
monthly_temp_data

---

### Methane Data <a id="methane-data"></a>

A composite record of historic methane data can be formed taking dats from multiple sources.

Methane data from 2023 to 1984 is obtained from the NOAA.

Methane data from 1982 to 1888 was sourced from the NOAA (Etheridge, Steele, Francey, and Langenfelds, 1998).

Methane data from from 1884 to 800,000 previous was sourced from the EPICA Dome C Ice Core dataset (Loulergue et al., 2008).

Methane levels are measured as CH4 mean (ppbv).

Methane data is read into Pandas DataFrames from the respective data sources

In [ ]:
methane_1 = pd.read_table('ch4_annmean_gl.txt', skiprows=45, sep=' ')
methane_2 = pd.read_table('law_ch4.txt', 
                        sep='\t', skiprows = 647, encoding='unicode_escape', names = ['year', 'ch4_ppb', 'Unnamed: 2', 
                                                                                      'Unnamed: 3', 'Unnamed: 4', '64'])
methane_3 = pd.read_table('law_ch4.txt', 
                        sep='\t', skiprows = 155, nrows=15, encoding='unicode_escape', 
                        names=['year', 'ch4_ppb', '4', '5', '6', '7'])
methane_4 = pd.read_table('https://www1.ncdc.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc-ch4-2008.txt', 
                          sep = ' ', skipinitialspace=True, skiprows = 155, 
                          names=['depth', 'yr_bp', 'ch4_ppb', '4', '5', '6', '7'])



Unnecessary columns are removed from the respective DataFrames. 

In [ ]:
methane_1.drop(['#', 'year', 'Unnamed: 3', 'Unnamed: 5', 'mean', 'Unnamed: 7', 'Unnamed: 8', 
                'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'unc'], axis=1, inplace=True)
methane_2.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '64'], axis=1, inplace=True)
methane_3.drop(['4', '5', '6', '7'], axis=1, inplace=True)
methane_4.drop(['depth', '4', '5', '6', '7'], axis=1, inplace=True)


Columns in *methane_1* are renamed to logical names.

In [ ]:
cols = ['year', 'ch4_ppb']
methane_1.columns = cols

Create a year column and calculate the year for the *methane_4* DataFrame.

In [ ]:
methane_4['year'] = 1950 - methane_4['yr_bp']

*Years before present* columns are created in each of the DataFrames to calculate the number of years before 2023 that the samples are from. This is achieved by defining a function and passing the DatFrames to the function via a for loop.

In [ ]:
def years_before_present(sample):
    sample['years_before_2023'] = 2023 - sample['year']

In [ ]:
methane_1 = methane_1.sort_values('year', ascending=False)
methane_2 = methane_2.sort_values('year', ascending=False)
methane_3 = methane_3.sort_values('year', ascending=False)
methane_4 = methane_4.sort_values('year', ascending=False)

In [ ]:
sets = [methane_1, methane_2, methane_3, methane_4]

for set in sets: years_before_present(set)

In [ ]:
methane_4 = methane_4.reindex(columns=['year', 'ch4_ppb', 'years_before_2023', 'yr_bp'])

In [ ]:
frames_methane = [methane_1, methane_2, methane_3, methane_4]

full_methane_data = pd.concat(frames_methane, ignore_index = True)

In [ ]:
full_methane_data.head()

The unneeded yr_bp column is dropped from the *full_temp* DataFrame

In [ ]:
full_methane_data.drop('yr_bp', axis=1, inplace=True)

The columns in the DataFrame are reordered.

In [ ]:
full_methane_data = full_methane_data.reindex(columns=['years_before_2023', 'year', 'ch4_ppb'])

Duplicate *year* variable entries are removed.

In [ ]:
full_methane_data.drop_duplicates(subset='years_before_2023', keep='first', inplace=True)

NA values are dropped from the DataFrame

In [ ]:
full_methane_data.dropna(inplace=True)

The variable Dtypes are explored.

In [ ]:
full_methane_data.info()

The year variables are cast to integers.

In [ ]:
full_methane_data = full_methane_data.astype({'year':'int', 'years_before_2023':'int'})

In [ ]:
full_methane_data.info()

In [ ]:
full_methane_data.head()

---

### Irish Data <a id="irish_data"></a>

Historic Irish temperature and observed precipitation data was sourced from the [World Bank Group](https://climateknowledgeportal.worldbank.org/) (https://climateknowledgeportal.worldbank.org/ Accessed 20 Dec. 2023) website who sourced their [data](https://climateknowledgeportal.worldbank.org/country/ireland/climate-data-historica) (https://climateknowledgeportal.worldbank.org/country/ireland/climate-data-historical Accessed 20 Dec. 2023) from  the climate research unit in the University of East Anglia (https://www.uea.ac.uk/groups-and-centres/climatic-research-unit Accessed 20 Dec. 2023).

A monthly data set of weather observations from Dublin Airport for the period of 1942 to 2003 was obatined from [Met Eireann](https://www.met.ie/about-us) (https://www.met.ie/about-us). This data was sourced from [here](https://www.met.ie/climate/available-data/historical-data) (https://www.met.ie/climate/available-data/historical-data).

#### Irish Temperature Data <a id="irish_temperature_data"></a>

Read in historic Irish annnual mean temperature data and view the first five lines of the dataset to get an overview  the variables in the dataset.

In [ ]:
irish_temp_data = pd.read_csv('observed_annual_average_mean_surface_air_temperature_of_ireland_for_1901_2022.csv')

In [ ]:
irish_temp_data.head()

The columns in the dataset are renamed to Python friendly names.

In [ ]:
irish_temp_data.rename(columns={'Category': 'year', 'Annual Mean': 'annual_mean', '5-yr smooth': '5_yr_smooth'}, inplace=True)

The data types are understood using the Pandas `info()` method.

In [ ]:
irish_temp_data.info()

Create a time series using the year as the index.

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html [Accessed 7 Nov. 2023]
irish_temp_data['time_series'] = pd.to_datetime(irish_temp_data['year'], format='%Y')

# set time_series column as index
# https://stackoverflow.com/questions/27032052/how-do-i-properly-set-the-datetimeindex-for-a-pandas-datetime-object-in-a-datafr
# [Accessed 7 Nov. 2023].
irish_temp_data = irish_temp_data.set_index('time_series')

A column is added to calculate the median of the *annual_mean* variable.

In [ ]:
irish_temp_data['median'] = irish_temp_data['annual_mean'].median()

In [ ]:
irish_temp_data

Read in monthly Irish weather data from Dublin Airport as a Pandas DataFrame.

In [ ]:
da_monthly_weather_data = pd.read_csv('mly532.csv', skiprows = 19)

Print the first five rows of the DataFrame to get an overview. The month and year variables are in separate columns. To create a time series these to variables need to be combined.

In [ ]:
da_monthly_weather_data.head(5)

To combine the year and month variables they are appended to a list and added into the DataFrame in the *date* column.

In [ ]:
# adapted from code found at 
# - https://stackoverflow.com/questions/48304927/cleanly-combine-year-and-month-columns-to-single-date-column-with-pandas 
# [Accessed 21 Dec. 2023].

DATE = []
for y, m in zip(da_monthly_weather_data.year, da_monthly_weather_data.month):
    DATE.append(date(y, m, 1))
    
da_monthly_weather_data['date'] = DATE

In [ ]:
da_monthly_weather_data.info()

In [ ]:
da_monthly_weather_data['time_series'] = pd.to_datetime(da_monthly_weather_data['date'], format='%d/%m/%Y')

da_monthly_weather_data = da_monthly_weather_data.set_index('time_series')

In [ ]:
da_monthly_weather_data

#### Irish Precipitation Data <a id="irish_temperature_data"></a>

Historic Irish precipitation data from the [World Bank Group](https://climateknowledgeportal.worldbank.org/) (https://climateknowledgeportal.worldbank.org/ Accessed 20 Dec. 2023) is read in as a Pandas DataFrame. This data gives an overview of the trend in precipitation lecvels from 1900 to 2022.

In [ ]:
irish_precip_data = pd.read_csv('observed_annual_precipitation_of_ireland_for_1901_2022.csv')

In [ ]:
irish_precip_data.head()

The column names are renamed to python friendly names.

In [ ]:
irish_precip_data.rename(columns={'Category': 'year', 'Annual Mean': 'annual_mean', '5-yr smooth': '5_yr_smooth'}, inplace=True)

The variables datatypes are explored using the Pandas `info()` method.

In [ ]:
irish_precip_data.info()

A time series is created using the year as the index.

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html [Accessed 7 Nov. 2023]
irish_precip_data['time_series'] = pd.to_datetime(irish_precip_data['year'], format='%Y')

# set time_series column as index
# https://stackoverflow.com/questions/27032052/how-do-i-properly-set-the-datetimeindex-for-a-pandas-datetime-object-in-a-datafr 
# [Accessed 7 Nov. 2023].
irish_precip_data = irish_precip_data.set_index('time_series')

In [ ]:
irish_precip_data

Historical monthly precipitation data from 1711 to 2016 is read in as a Pandas DataFrame.

In [ ]:
irish_monthly_precip_data = pd.read_csv('IOI_1711_SERIES.CSV')

In [ ]:
irish_monthly_precip_data

The columns are renamed to python friendly names.

In [ ]:
irish_monthly_precip_data.rename(columns={'Year': 'year', 'Month': 'month', 'Median montly series': 'median_monthly_precip_mm'},
                                  inplace=True)

In [ ]:
irish_monthly_precip_data.head()

The month and year columns are merged so that a time series can be created.

In [ ]:
DATE = []
for y, m in zip(irish_monthly_precip_data.year, irish_monthly_precip_data.month):
    DATE.append(date(y, m, 1))
    
irish_monthly_precip_data['date'] = DATE

A time series is set as the index of the DataFrame.

In [ ]:
irish_monthly_precip_data['time_series'] = pd.to_datetime(irish_monthly_precip_data['date'], format='%d/%m/%Y')

irish_monthly_precip_data = irish_monthly_precip_data.set_index('time_series')

---

### Fused Dataset <a id="fused_dataset"></a>

A fused dataset consisting of historical carbon dioxide, temperature and methane levels can be created by merging the DataFrames containing these variables these into one Pandas DataFrame. In order to do this the relevant data must be selected.

For historical temperature data the IPCC dataset will be used as it uses the AICC2012 system to date the carbon dioxide samples in the dataset. The AICC2012 system is expanded upon in the [analysis](#analysis) section of this notebook. The reason for selecting this is that it said to provide an improved timing for the bipolar nature of the interglacial periods.

Temperature data from the NOAA is dated using the EDC3 system. If this was used with the carbon dioxide data from the IPCC there would be a mismatch between the timings of the samples as these two systems are not in agreement with each other as to the age of the ice core samples. This is demonstrated in the [analysis](#analysis) section by plotting two sets of carbon dioxide that have been aged using each of these systems. As [demonstrated](#carbon-dioxide) the trends on the graphs are nearly identical but the timings of the data points are out of step with each other. For this reason temperature data that was sourced from [NASA](https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt) (https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt [Accessed 12 Dec. 2023]) and [https://www.temperaturerecord.org/#sources](https://www.temperaturerecord.org/#sources), accessed 13 Dec. 2023 is used in the fused dataset.

Finally the methane data from [here](#methane-data) is used in the fused dataset.

IPCC carbon dioxide data is concatenated to the most recent co2 data from the Mauna Loa observatory.

In [ ]:
frames = [mauna_loa_co2, ipcc_full_co2_data]

full_co2_data = pd.concat(frames, ignore_index = True)

In [ ]:
full_co2_data.head()

Inspecting the DataFrame it can be seen that the *year* and *years_before_2023* variables are integers and the *co2_ppmv* variables is a floating point integer.

In [ ]:
full_co2_data.info()

The full carbon dioxide dataset is merged into the full temperature dataset on the *year* variable.

In [ ]:
# code adapted from https://stackoverflow.com/questions/53645882/pandas-merging-101 [Accessed 18 Dec. 2023]
fused_data = full_co2_data.merge(full_temp_data, on = 'years_before_2023', how = 'outer') 

In [ ]:
fused_data.head()

Further operations are carried out on the fused dataset to remove unnecessary columns.

In [ ]:
fused_data.drop(['year_x', 'year_y'], axis=1, inplace=True)

Duplicates are checked in the *fused_data* DataFrame.

In [ ]:
pd.concat(g for _, g in fused_data.groupby("years_before_2023") if len(g) > 1)

The single duplicate value is dropped.

In [ ]:
fused_data.drop_duplicates(subset='years_before_2023', inplace=True)

The data is sorted by descending year and the ordering of the columns is changed.

In [ ]:
fused_data = fused_data.sort_values('years_before_2023', ascending=True).reset_index(drop=True)

The full methane dataset variables are checked.

In [ ]:
full_methane_data.info()

The *full methane data* DataFrame is merged into the *fused_data* DataFrame.

In [ ]:
fused_data = fused_data.merge(full_methane_data, on='years_before_2023', how='outer')

The data in the *fused data* DataFrame is sorted by descending year.

In [ ]:
fused_data = fused_data.sort_values('years_before_2023', ascending=True).reset_index(drop=True)

The unnecessary year column is dropped from the *fused_data* DataFrame.

In [ ]:
fused_data.drop('year', axis=1, inplace=True)

In [ ]:
fused_data.isnull().sum()

In [ ]:
fused_data.interpolate(inplace=True)

In [ ]:
fused_data.to_excel('fused.data.xlsx')

Exploring the variables in the *fused_data* demonstrates that there are 5 vairiables in this dataset.

In [ ]:
fused_data.info()

The variables can be summarised as follows:

Applying variable classification and level of measurement the dataset can be summarised as follows:

**Fused Dataset Variable Summary Table**

| Variable              | Classification    | Type          | Python Dtype  | Unit of Measure                                           | Level of Measure  |
| :----------           | :----------       | :----------   | :----------   | :---------                                                |:--------          |
| year                  | numerical         | interval      | int64         | year                                                      | interval          |
| co2_ppmv              | numerical         | continuous    | float64       | parts per million by volume                               | ratio             |
| temp_anomaly          | numerical         | continuous    | float64       | degrees celsius                                           | interval          |
| ch4_ppb               | numerical         | continuous    | float64       | mole fraction in dry air, nanomol/mol, abbreviated as ppb | ratio             |
| years_before_present  | numerical         | interval      | int64         | degrees celcius                                           | interval          |

There will be null values in the fused dataset due to the fact that the dates the measurements are aged to do not always match.

Checking the null values confirms this. It is important that the *year* and *years_before_present* variables do not have any null values as this confirms that there is at least one data entry for the other variables in each row of the dataset.

In [ ]:
fused_data.isnull().sum()

In [ ]:
fd_corr = fused_data.corr(numeric_only=True)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(fd_corr)

## Analysis <a id="analysis"></a>

### Carbon Dioxide <a id="carbon_dioxide"></a>



A composite of the IPCC data is plotted. This plot demonstrates approximately 800,000 years of co2 data. The composite graph demonstrates how the results from each study have been stitched together to form a cohesive narrative on the fluctuations of  carbon dioxide levels over the last 800,000 years. The plot demonstrates peaks and lows of carbon dioxide approximately every 100,000 years. This aligns with glacial-interglacial cycles occurring every 100,000 years since the Quaternary period (Lisiecki and Raymo, 2005).) During these periods, as polar ice caps melt carbon dioxide is released into the atmosphere thus causing the the rise in atmospheric carbon dioxide.

A rapid rise in carbon dioxide levels during the 21st century is observed. This rise is greater and more rapid than can be accounted for by the glacial-interglacial cycle and is attributed to the post industrial revolution large scale usage of fossil fuels.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15,10))

# plot the composite graph with labelling and titles
ax.plot(rubino['years_before_2023'], rubino['co2_ppmv'], label='Law Dome (Rubino et al., 2013)')
ax.plot(macfarling['years_before_2023'], macfarling['co2_ppmv'], label = 'Law Dome (MacFarling Meure et al., 2006)')
ax.plot(monnin['years_before_2023'], monnin['co2_ppmv'], label = 'Dome C (Monnin et al., 2001 + 2004)')
ax.plot(marcott['years_before_2023'], marcott['co2_ppmv'], label = 'WAIS (Marcott et al., 2014)')
ax.plot(ahn['years_before_2023'], ahn['co2_ppmv'], label = 'Siple Dome (Ahn et al., 2014)')
ax.plot(bereiter['years_before_2023'], bereiter['co2_ppmv'], label = 'TALDICE (Bereiter et al., 2012)')
ax.plot(bereiter_2['years_before_2023'], bereiter_2['co2_ppmv'], label  = 'EDML (Bereiter et al., 2012)')
ax.plot(schneider['years_before_2023'], schneider['co2_ppmv'], label = 'Dome C Sublimation (Schneider et al., 2013)')
ax.plot(petit['years_before_2023'], petit['co2_ppmv'], label = 'Vostok (Petit et al., 1999)')
ax.plot(siegenthaler['years_before_2023'], siegenthaler['co2_ppmv'], label = 'Dome C (Siegenthaler et al., 2005)')
ax.plot(siegenthaler_2['years_before_2023'], siegenthaler_2['co2_ppmv'], label = 'Dome C (Siegenthaler et al., 2005)')
ax.plot(bereiter_3['years_before_2023'], bereiter_3['co2_ppmv'], label = 'Dome C (Bereiter et al., 2014)')
ax.set_xlabel('Years Before 2023')
ax.set_ylabel('c02 PPM')
ax.legend()
plt.suptitle("IPCC - 800k Years of co2", weight = 'bold', size = 15)
plt.tight_layout()

The composite of the Luthi et al subsets of data is plotted in the same way as the IPCC composite data. An immediate observation is that the data is not as well joined as the data in the IPCC. This is observed at the intersection between Dome C Siegenthaler and Dome C Luthi. The intersections of the subsets of data in the IPCC set are joined less crudely than Luthi. Another noticeable observation is that the large spike in carbon dioxide from the modern era is not observed in Luthi's dataset, most likely as the first data point is in 1813, before the widespread usage of fossil fuels.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15,10))

# plot the composite graph with labelling and titles
ax.plot(monnin_luthi['years_before_2023'], monnin_luthi['co2_ppmv'], label='Dome C (Monnin et al. 2001)')
ax.plot(pettit_luthi['years_before_2023'], pettit_luthi['co2_ppmv'], label = 'Vostok (Petit et al. 1999; Pepin et al. 2001; Raynaud et al. 2005)')
ax.plot(siegenthaler_1_luthi['years_before_2023'], siegenthaler_1_luthi['co2_ppmv'], label = 'Dome C (Siegenthaler et al. 2005)')
ax.plot(siegenthaler_2_luthi['years_before_2023'], siegenthaler_2_luthi['co2_ppmv'], label = 'Dome C (Siegenthaler et al. 2005)')
ax.plot(luthi_luthi['years_before_2023'], luthi_luthi['co2_ppmv'], label = 'Siple Dome (Dome C (Luethi et al. (sub)')
ax.set_xlabel('Years Before Present')
ax.set_ylabel('c02 PPM')
ax.legend()
plt.suptitle("Luthi - 800k Years of co2", weight = 'bold', size = 15)
plt.tight_layout()

Plotting both carbon dioxide datasets over one another on the same axis allows for a direct comparison.

The most noticeable feature is that the two plots create a blurred appearance as if they are slightly out of phase with one another.

The IPCC data uses the The Antarctic Ice Core Chronology (AICC2012) (Veres et al, 2013) system to age carbon dioxide samples. The data in Luthi dataset uses the EDC3 chronology (Parrenin et al., 2007). The use of two different systems to data the same ice core samples would account for the discrepancy in datapoints that have the same co2 ppm concentration but are slightly out of synchronisation in terms of the time from present when the datapoints fall.

The difference between the two systems is that the AICC2012 timescale has been constructed using the Bayesian tool Datice (Lemieux-Dudon et al., 2010). The Datice tool combines glaciological inputs and data constraints. Whereas the EDC3 timescale is based on the use of a snow accumulation and mechanical flow model and a set a set of independent age markers along the core (Parrenin et al, 2007).

AICC2012 is said to present an improved timing for the last glacial inception and presents a slightly improved timing for the bipolar sequence of events over Marine Isotope Stage 3 (Bazin et al., 2013). Given the see saw nature of glacial-interglacial periods this suggests that the AICC2012 provides a better resolution on the age of samples.

Basin et al (op. cit.) found that the biggest deviation between AICC2012 and EDC occurs around Marine Isotope Stage 12 (MIS 12) approximately 478K years ago. This is not apparent on the plot below. It appears that there is a noticeably bigger deviation between 350,000 - 400,00 years ago.

Some parts of the respective plots cancel each other out (from approximately 600,000-700,00 years ago) suggesting very little disparity between the two chronological systems for samples that fall in that age range.

Apart from the period of approximately 20,00 years ago to 100,000 years ago the trends over time on the two datasets closely match one another. 

It can be said that both datasets are largely in agreement over the trends in atmospheric carbon dioxide over time but that there is disagreement over when these fluctuations occurred.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

ax.plot(ipcc_full_co2_data['years_before_2023'], ipcc_full_co2_data['co2_ppmv'], label = 'IPCC 800k Years of co2 Data')
ax.plot(luthi_full_co2_data['years_before_2023'], luthi_full_co2_data['co2_ppmv'], label = 'Luthi 800k Years of co2 Data', color = 'black', alpha = 0.6)
ax.set_xlabel('Years Before Present')
ax.set_ylabel('c02 PPM')
ax.legend()
plt.suptitle('IPCC v Luthi 800k Years of co2 Data', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()

Plotting the trend of atmospheric carbon dioxide levels to include data from the last 20 years, from Mauno Loa, visualises the large jump in atmospheric CO2 in the 21st century. It should be noted that the data from Mauno Loa, located in the Pacific is from a different geographical location than that of the Antartic. This may be exaggerating the jump that is seen in the CO2 levels due to the fact that the measurements are drawn different geographic locations.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 7))

# plot the results
ax.plot('years_before_2023', 'co2_ppmv', data=fused_data)
ax.set_xlabel('Years Before 2023')
ax.set_ylabel('CO2 PPM')
ax.set_title('800k Years of co2 Data From 2022', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()

Inspecting the frequency distribution (histogram) of the IPCC carbon dioxide data values it can be see that, in the 800,000 year record, atmospheric levels of co2 over 300 ppm occur infrequently. This suggests the spike in atmospheric carbon dioxide occurring in the last 200 years is an an event that falls outside of the cyclical rise and fall of carbon dioxide during glacial-interglacial periods.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(8,5))

# plot the results
ax.hist(fused_data['co2_ppmv'])
ax.set_xlabel('co2_ppmv')
ax.set_ylabel('Count')
ax.set_title('IPCC Co2 Data Histogram', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()

Plotting the box plot for the full carbon dioxide dataset it can be seen that carbon dioxide levels above 350 ppmv are outliers and thus are not a normal occurrence. An outlier is described described as an object that deviate significantly from the rest for the objects. This strengthens the hypothesis that the rise in atmospheric CO2 levels in the 21st century is a new phenomenon that is not related to the glacial-interglacial rise and fall of atmospheric carbon dioxide levels.

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y = 'co2_ppmv', data = full_co2_data, color = 'green').set_title('IPCC 800k Years of CO2 Boxplot', weight = 'bold', size = 15)

Monthly CO2 levels since 1979 are available from from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.txt (Accessed 23 Dec. 2023). Since 1979 there is a clear upward trend CO@ levels. However, while the overall levels are trending upwards the amount of CO2 in the atmosphere appears to fluctuate up and down over time. This indicates that there is a seasonality to the amount of Co2 present in the atmosphere.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15,7))

# plot the results
ax.plot(monthly_co2_data['average'])
ax.set_xlabel('Year/Month')
ax.set_ylabel('CO2 PPM')
ax.set_title('Monthly CO2 Trends Since 1979', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()



The seasonal variation i n the signal can be understood by de-trending it.

The plot below demonstrates the original signal with the trend line overlaid. Beneath that the deconstructed signal is plotted on a subplot.

In [ ]:
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# remove unneeded coluns from the dataset.
monthly_co2_data.drop(['year', 'month'], axis=1, inplace=True)

In [ ]:
decomposition = seasonal_decompose(monthly_co2_data['average'])
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid


In [ ]:
# based on code found at https://ataspinar.com/2020/12/22/time-series-forecasting-with-stochastic-signal-analysis-techniques/ Accessed 28 Dec. 2023

# plot size and title
plt.figure(figsize=(18, 6))

# plot the visualisation boxplot
plt.subplot(2, 1, 1)
sns.lineplot(data = monthly_co2_data['average']).set_title('Original Signal & Trend')
sns.lineplot(trend)

plt.subplot(2, 1, 2)
sns.lineplot(seasonal).set_title('De-trended Signal')


plt.suptitle('Monthly CO2 Variation Since 1979', fontweight ="bold", fontsize=18)
plt.tight_layout()

Once decomposed the signal can be plotted in the frequency domain. Viewing the signal in this way allows shows how much if the signal lies within each given frequency band. The is achieved by transforming the data to the frequency domain using the Fast Fourier Transform (FFT).

The FFT transforms the data from the time domain to the frequency domain. FFT is widely used in a number of fields such as engineering, science, music and mathematics.

In [ ]:
# import the required functions
from scipy.signal import savgol_filter
from siml.detect_peaks import *


To plot the signal in the frequency domain the signal is de-trended and transformed using the the FFT functions in Numpy.

In [ ]:
yvalues = monthly_co2_data['average'].values
yvalues_trend = savgol_filter(yvalues,25,1)
yvalues_detrended = yvalues - yvalues_trend

fft_y_  = np.fft.fft(yvalues_detrended)
fft_y = np.abs(fft_y_[:len(fft_y_)//2])
indices_peaks = detect_peaks(fft_y, mph=25)
 
fft_x_ = np.fft.fftfreq(len(yvalues_detrended))
fft_x = fft_x_[:len(fft_x_)//2]

The signal is plotted in the frequency domain. 

The plot shows spikes in signal amplitude at f = 0.08, f = 0.17 and f = o.25

$\implies 1 \div 0.08 \approx 12.5$  months

$\implies 1 \div 0.17 \approx 5.88$ months

$\implies 1 \div 0.25 = 4$ months

All of the above indicates that there is a seasonal occurrence that happens yearly, every six months and every quarter. This can be explained by the 12 month gaps between the respective highs and lows of summer winter high, the six month gap between the winter high and summer low and the quarterly changes between each of the four seasons.

In [ ]:
# create an empty plot  
fig, ax = plt.subplots(figsize=(15,7))

# plot the results
ax.plot(fft_x, fft_y)
ax.scatter(fft_x[indices_peaks], fft_y[indices_peaks], color='red',marker='D')
ax.set_title('Frequency Spectrum of Monthly CO2 Emissions', fontsize=15, weight = 'bold')
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_xlabel('Frequency [1/month]', fontsize=14)
 
 # annotate the peaks
for idx in indices_peaks:
    x,y = fft_x[idx], fft_y[idx]
    text = "  f = {:.2f}".format(x,y)
    ax.annotate(text, (x,y))
plt.show()

### Temperature <a id="temperature"></a>

Firstly, the data on which the temperature anomaly was manually calculated for (see [here](#jouzel-2007-temperature-data)) is plotted against the NOAA temperature data . The data that has a manually calculated temperature anomaly is very close to the data sourced from the NOAA, suggesting that the transformation done on that data is close to accurate.

Once again the two datasets appear "out of phase with one another". This effect, explained above, results from the disparities between the AICC2023 and EDC3 chronologies used to date the samples. The two chronologies are in agreement about the rising and falling CO2 levels but disgree on the age of the samples on the dataset.

As the full CO2 data set uses the AICC2023 chronology future analysis on ice core temperature data, in this notebook, will use this chronology so that there is a consistency in comparison of the two variables.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 10))

# plot the results
ax.plot(jouzel_temp['aicc2012_years_before_2023'], jouzel_temp['anomaly'], label = 'EPICA Dome C ice core AICC2023 Chronology')
ax.plot(noaa_temp['years_before_2023'], noaa_temp['delta_t_c'], label = 'EPICA Dome C ice core EDC3 Chronology', color = 'black', alpha = 0.6)
ax.set_xlabel('Years Before 2023')
ax.set_ylabel('Temperature Degrees Celsius')
ax.legend()
plt.suptitle('AICC2023 Chronology vs EDC3 Chronology - 800k Years of Temperature Anomaly', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()

For comparative purposes Jouzel's temperature data (AICC2012 chronology) and the temperature data sourced from [https://www.temperaturerecord.org/#sources](https://www.temperaturerecord.org/#sources) are plotted on the same axis. The data from https://www.temperaturerecord.org/#sources is a global temperature record as opposed to one from a specific location in the case of Jouzel's antartic data.

The first observation from the plots of the data is that the data from https://www.temperaturerecord.org/#sources has been scaled. As [outlined here](#further-temperature-data) this data has been scaled to a four degree range. The unscaled EPICA data has large swings in the in the interglacial periods which visually translates as very dramatic when compared to the global temperature anomaly from the scaled dataset.

The second notable feature on this plot is that trend line of the fluctuating temperature anomaly for both datasets closely matches each other. That is to say the both datasets are in agreement about the rising and falling temperature anomaly over time.

As with the plots of carbon dioxide levels over time, the glacial-interglacial periods are occurring approximately every 100,000 years as seen by the cyclical rise and fall of the temperature anomaly over approximately 100,000 year cycles.

The plots indicates that the earth was hotter just over 100,000 years ago than it is now. This suggests that the current period of global warming is not unprecedented. The plot also indicates that until recently the earth was cooling as part of the glacial-interglacial cycle. That trend has been reversed for some reason and the earth has started to heat outside of what would be expected during this cycle.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 10))

# plot the results
ax.plot(jouzel_temp['aicc2012_years_before_2023'], jouzel_temp['anomaly'], label = 'EPICA Dome C ice core AICC2023 Chronology')
ax.plot(full_temp_data['years_before_2023'], full_temp_data['temp_anomaly'], label =
        'Composite of Ocean margin sediment cores, lake and ice cores on land and sea surface temperature records.', 
        color ='black' , alpha = 0.6)
ax.set_xlabel('Years Before 2023')
ax.set_ylabel('Temperature Degrees Celsius')
ax.legend()
plt.suptitle('800k Years of Temperature Anomaly', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()

In [ ]:
# plot size
plt.figure(figsize=(14, 8))
plt.subplot(1, 1, 1)
sns.boxplot(data = jouzel_temp, y  = 'anomaly').set_title("Jouzel AICC2012 Chronology 800,000 Years of Antarctic Temperature Anomaly", weight = 'bold', size = 15)


In [ ]:
monthly_temp_data

In [ ]:
decomposition = seasonal_decompose(monthly_temp_data['mean'])
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid


In [ ]:
# plot size and title
plt.figure(figsize=(18, 6))

# plot the visualisation boxplot
plt.subplot(2, 1, 1)
sns.lineplot(data = monthly_temp_data['mean']).set_title('Original Signal & Trend')
sns.lineplot(trend)

plt.subplot(2, 1, 2)
sns.lineplot(seasonal).set_title('De-trended Signal')


plt.suptitle('Monthly Temperature Anomaly Variation Since 1880', fontweight ="bold", fontsize=18)
plt.tight_layout()

In [ ]:
# import the required functions
from scipy.signal import savgol_filter
from siml.detect_peaks import *

In [ ]:
yvalues = monthly_temp_data['mean'].values
yvalues_trend = savgol_filter(yvalues,25,1)
yvalues_detrended = yvalues - yvalues_trend

fft_y_  = np.fft.fft(yvalues_detrended)
fft_y = np.abs(fft_y_[:len(fft_y_)//2])
indices_peaks = detect_peaks(fft_y, mph=13)
 
fft_x_ = np.fft.fftfreq(len(yvalues_detrended))
fft_x = fft_x_[:len(fft_x_)//2]

In [ ]:
# create an empty plot  
fig, ax = plt.subplots(figsize=(15,7))

# plot the results
ax.plot(fft_x, fft_y)
ax.scatter(fft_x[indices_peaks], fft_y[indices_peaks], color='red',marker='D')
ax.set_title('Frequency Spectrum of Monthly Temperature Anomaly', fontsize=15, weight = 'bold')
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_xlabel('Frequency [1/month]', fontsize=14)
 
 # annotate the peaks
for idx in indices_peaks:
    x,y = fft_x[idx], fft_y[idx]
    text = "  f = {:.2f}".format(x,y)
    ax.annotate(text, (x,y))
plt.show()

### Methane <a id="methane"></a>


Plotting the methane trend for the last 800K years demonstates:

+ that there is a massive spike, in approximately the last 200 -300 years, in the level of atmospheric methane. The large spike in this period begs the question is this spike is real or is it the result of bad quality data or some other underlying cause. To answer this question - *"Over the last two centuries, methane concentrations in the atmosphere have more than doubled, largely due to human-related activities."* (US EPA, 2022).

+ excluding approximately the last 500 years there is a cyclical rise and fall in methane levels that corresponds with glacial - interglacial periods in earths recent history. 

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 10))

# plot the results
ax.plot(full_methane_data['years_before_2023'], full_methane_data['ch4_ppb'])

ax.set_xlabel('Years Before 2023')
ax.set_ylabel('ch4_ppb')
ax.legend()
plt.suptitle('800k Years of Methane Trend', weight = 'bold', size = 15)
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(full_methane_data['ch4_ppb'])

### Correlation <a id="correlation"></a>

## Irish Context <a id="irish-context"></a>

### Irish Context - Temperature <a id="irish-context---temperature"></a>

A time series plot of the Irish temperature data is plotted. The trend line displays the annual variation in mean temperature year on year. Because of the annual variations in this variable the signal can said to be noisy. Plotting the median annual temperature visualises the years in which fell above or below this value. Of note since 2000 there has been only one year that has had an annual median temperature below the median value suggesting that the years in this period have been warmer than the years to this point.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15,10))

# plot the results
ax.plot(irish_temp_data['annual_mean'])
ax.plot(irish_temp_data['median'], label = 'median')
ax.set_ylabel('Degrees Celsius')
plt.suptitle('Irish Annual Mean Temperature', weight = 'bold', size = 15)
ax.legend()
plt.tight_layout()
plt.show()

The distribution of the mean annual temperature for the last 122 years is plotted. The distribution of data is close to normal. There is a spike in the frequency of datapoints around ten degrees celsius.

In [ ]:
fig, ax =plt.subplots()

ax.hist(irish_temp_data['annual_mean'])
ax.set_xlabel('Degrees Celsius')
ax.set_ylabel('Count')
ax.set_title('Irish Annual Mean Temperature Distribution 1900 - 2023', weight = 'bold', size = 15)
plt.tight_layout()


plt.show()

Using a time stepped linear regression it is possible to plot the relationship between time and the average annual mean temperature. Plotting the regression line smooths the variation in annual mean temperature. The line of regression indicates that over the 120 year period since 1900 there has been an increase in the annual mean temperature of approximately 0.8 of a degree celsius.

In [ ]:
irish_temp_data['time'] =  np.arange(len(irish_temp_data.index))

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.plot('time', 'annual_mean', data=irish_temp_data, color='0.75')
ax = sns.regplot(x='time', y='annual_mean', data=irish_temp_data, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of Irish Annual Mean Temperature 1900- 2022', weight = 'bold', size = 15)

Murphy et al. (2023) found that annual mean temperature with warming for the island of Ireland was estimated at 0.88 °C per degree increase in global mean surface temperature. These authors argued that the change in temperature in Ireland has been relatively small and therefore hard to perceive. The increase is more obvious during summer months.

Plotting global temperature anomaly against the Irish annual mean temperature from 1900 to 2022 confirms that both variables are trending in the same direction and that Ireland is getting warmer as the globe temperature increases.

To plot this data the relevant global data is assigned to a DataFrame.

In [ ]:
gta = full_temp_data.copy()

The relevant time period is located within the DataFrame.

In [ ]:
gta = gta.iloc[0:122]

The data is sortd by ascending year.

In [ ]:
gta = gta.sort_values('year', ascending=True)

A time series index is added to the dataset.

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html [Accessed 21 Dec. 2023]
gta['time_series'] = pd.to_datetime(gta['year'], format='%Y')

# set time_series column as index
# https://stackoverflow.com/questions/27032052/how-do-i-properly-set-the-datetimeindex-for-a-pandas-datetime-object-in-a-datafr [Accessed 21 Dec. 2023].
gta = gta.set_index('time_series')

A step is added to the DataFrame to match the Irish Temperature Data.

In [ ]:
gta['time'] =  np.arange(len(gta.index))

The graph is plotted.

In [ ]:
# adapted from code found here - https://stackoverflow.com/questions/5484922/secondary-axis-with-twinx-how-to-add-to-legend Accessed 21 Dec. 2023

# create an empty plot
fig, ax = plt.subplots(figsize=(15,10))

g1 = ax.plot('time', 'annual_mean', data = irish_temp_data, color='0.75', label = 'Irish Annual Mean Temperature')
ax2 = ax.twinx()
g2 = ax2.plot('time', 'temp_anomaly', data = gta, label = 'Global Temperature Anomaly')
ax.set_title('Global Temperature Anomaly V Irish Annual Mean Temperature 1900-2022', weight='bold', size=15)

# display legends
lns = g1+g2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=0)

# add labels
ax.set_xlabel("Years From 1900")
ax.set_ylabel('Irish Annual Mean Temperature Degrees Celsius')
ax2.set_ylabel("Temp Anomaly Degrees Celsius")

#plt.suptitle('Global Temperature Anomaly V Irish Annual Mean Temperature 1900-2022 ', weight='bold', size=15)
plt.tight_layout()

The two variables can be tested to understand if there is a lag between them. In this case the *y variable*, the mean annual Irish temperature, lags the *x variable* by 0. This indicates that the rise in temperature in Ireland happens at the same time as the rise in the global temperature anomaly.

In [ ]:
# Adapted code from - https://stackoverflow.com/questions/69117617/how-to-find-the-lag-between-two-time-series-using-cross-correlation Accessed 21 Dec. 2023
x = gta['temp_anomaly']
y = irish_temp_data['annual_mean']
correlation = signal.correlate(x-np.mean(x), y - np.mean(y), mode="full")
lags = signal.correlation_lags(len(x), len(y), mode="full")
lag = lags[np.argmax(abs(correlation))]

In [ ]:
print(lag)

The data analysed above is annual data. Weather variables are subject to seasonal variation. For example, Harmel et al. (2002) and Ruff and Neelin (2012) provide evidence that the distribution of daily temperature data is skewed and that the skewness depends on the season. Temperature for each of the seasons can be anlaysed over time to test the assumption that temperature increases are hard to perceive and seasonal. The data analysed in this section examines data from the Dublin airport weather station. It is recognised that this weather data is from one location and differs from the all Ireland data used by Murphy et al (2023). However, this data can act as a a proxy for national data.

As per Wikipedia (Wikipedia Contributors, 2023), Ireland experiences 4 seasons:

- Winter = November, December & January
- Spring = February, March & April
- Summer = May, June & July
- Autumn = August, September & October

These are the definitions of season that are used in the following section.

Plotting boxplots of the Irish monthly temperature in Dublin Airport demonstrates how the median temperature changes month by month.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 10))

# plot the boxplots
sns.boxplot(x='month', y='meant', data =da_monthly_weather_data).set_title('Monthly Mean Temperature Variation Dublin Airport 1941-2023', weight='bold', size=15)

Season specific DataFrames of monthly weather data from Dublin airport are assigned.

In [ ]:
# Filter DateFrame by seasons - https://kanoki.org/2022/07/16/pandas-filter-dates-by-month-hour-day-or-last-n-days-weeks/ [Accessed 8 Nov. 2023]. 
winter = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['November', 'December', 'January'])].sort_index(ascending=True)
spring = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['February', 'March', 'April'])].sort_index(ascending=True)
summer = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['May', 'June', 'July'])].sort_index(ascending=True)
autumn = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['August', 'September', 'October'])].sort_index(ascending=True)

Plotting the data points without smoothing will result in a plot that is difficult to interpret due to the monthly variation in mean temperature values. Thge type of smoothin applied is moving average smoothing. This type of smoothing is a common type of smoothing used in time series analysis.

In [ ]:
# code adapted from https://medium.com/@srv96/smoothing-techniques-for-time-series-data-91cccfd008a2 Accessed 21 Dec. 2023
def moving_avarage_smoothing(X,k):
	S = np.zeros(X.shape[0])
	for t in range(X.shape[0]):
		if t < k:
			S[t] = np.mean(X[:t+1])
		else:
			S[t] = np.sum(X[t-k:t])/k
	return S

The plot demonstrates that in spring in winter the mean monthly temperatures have been consistent since 1942 without any obvious dramatic upward trend. Summer and Autumn have been gradually getting warmer. Taking this into account it is consistent with Murphy et al. (2023) in so far as the gradual increase in temperature is relatively small and confined to summer and autumn months. There could be a perception that winter feels colder however the data suggests that the temperature that over the last 80 years there is little variation in the mean monthly temperature.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

ax.plot(moving_avarage_smoothing(winter['meant'], 250), label = 'Winter Mean Temperature', color = 'blue')
ax.plot(moving_avarage_smoothing(spring['meant'], 250), label = 'Spring Mean Temperature', color = 'black')
ax.plot(moving_avarage_smoothing(summer['meant'], 250), label = 'Summer Mean Temperature', color = 'red')
ax.plot(moving_avarage_smoothing(autumn['meant'], 250), label = 'Autumn Mean Temperature', color = 'grey')
ax.set_xlabel("No of Months")
ax.set_ylabel('Degrees Celsius')
ax.legend()
plt.suptitle('Season Months Mean Temperature 1942 to 2023 - Dublin Airport', weight='bold', size=15)
plt.tight_layout()



In [ ]:
da_monthly_weather_data_1980 = da_monthly_weather_data['1980':]['meant']
da_monthly_weather_data_1941 = da_monthly_weather_data['1941':'1980']['meant']

In [ ]:
da_monthly_weather_data_1941

In [ ]:
decomposition = seasonal_decompose(da_monthly_weather_data_1980)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

decomposition_1941 = seasonal_decompose(da_monthly_weather_data_1941)
trend_1941 = decomposition_1941.trend
seasonal_1941 = decomposition_1941.seasonal
residual_1941 = decomposition_1941.resid

In [ ]:
# plot size and title
plt.figure(figsize=(18, 6))

# plot the visualisation boxplot
plt.subplot(2, 1, 1)
sns.lineplot(data = da_monthly_weather_data_1980).set_title('Original Signal & Trend')
sns.lineplot(trend)

plt.subplot(2, 1, 2)
sns.lineplot(data = da_monthly_weather_data_1941).set_title('Original Signal & Trend')
sns.lineplot(trend_1941)

plt.suptitle('Irish Mean Monthly Temperature Variation Since 1980', fontweight ="bold", fontsize=18)
plt.tight_layout()

In [ ]:
# plot size and title
plt.figure(figsize=(18, 6))

plt.subplot(2, 1, 1)
sns.lineplot(seasonal).set_title('De-trended Signal')

plt.subplot(2, 1, 2)
sns.lineplot(seasonal_1941).set_title('De-trended Signal')



In [ ]:
yvalues = da_monthly_weather_data_1980.values
yvalues_trend = savgol_filter(yvalues,25,1)
yvalues_detrended = yvalues - yvalues_trend

fft_y_  = np.fft.fft(yvalues_detrended)
fft_y = np.abs(fft_y_[:len(fft_y_)//2])
indices_peaks = detect_peaks(fft_y, mph=50)
 
fft_x_ = np.fft.fftfreq(len(yvalues_detrended))
fft_x = fft_x_[:len(fft_x_)//2]

In [ ]:
yvalues_1941 = da_monthly_weather_data_1941.values
yvalues_trend_1941 = savgol_filter(yvalues_1941,25,1)
yvalues_detrended_1941 = yvalues_1941 - yvalues_trend_1941

fft_y_1941  = np.fft.fft(yvalues_detrended_1941)
fft_y1941 = np.abs(fft_y_[:len(fft_y_1941)//2])
indices_peaks_1941 = detect_peaks(fft_y1941, mph=50)
 
fft_x_1941 = np.fft.fftfreq(len(yvalues_detrended_1941))
fft_x1941 = fft_x_1941[:len(fft_x_1941)//2]

In [ ]:
# create an empty plot  
fig, ax = plt.subplots(figsize=(15,7))

# plot the results
ax.plot(fft_x, fft_y)
ax.scatter(fft_x[indices_peaks], fft_y[indices_peaks], color='red',marker='D')
ax.set_title('Frequency Spectrum of Irish Monthly Mean Temperature', fontsize=15, weight = 'bold')
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_xlabel('Frequency [1/month]', fontsize=14)
 
 # annotate the peaks
for idx in indices_peaks:
    x,y = fft_x[idx], fft_y[idx]
    text = "  f = {:.2f}".format(x,y)
    ax.annotate(text, (x,y))
plt.show()

In [ ]:
# create an empty plot  
fig, ax = plt.subplots(figsize=(15,7))

# plot the results
ax.plot(fft_x1941, fft_y1941)
ax.scatter(fft_x1941[indices_peaks], fft_y1941[indices_peaks], color='red',marker='D')
ax.set_title('Frequency Spectrum of Irish Monthly Mean Temperature', fontsize=15, weight = 'bold')
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_xlabel('Frequency [1/month]', fontsize=14)
 
 # annotate the peaks
for idx in indices_peaks:
    x,y = fft_x1941[idx], fft_y1941[idx]
    text = "  f = {:.2f}".format(x,y)
    ax.annotate(text, (x,y))
plt.show()

### Irish Context - Precipitation <a id="irish-context---precipitation"></a>

In [ ]:
irish_precip_data

A column is added to calculate the median value of the annual mean precipitation.

In [ ]:
irish_precip_data['median'] = irish_precip_data['annual_mean'].median()

Annual mean precipitation time series data is plotted. The normal annual variation in precipitation is visualised. It is difficult to discern any trend or pattern in this plot as the signal varies year on year.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15,10))

# plot the results
ax.plot(irish_precip_data['annual_mean'])
ax.plot(irish_precip_data['median'], label = 'Median of the Mean Values')
ax.set_ylabel('Precipitation mm')
plt.suptitle('Irish Annual Mean Precipitation 1901 - 2022', weight = 'bold', size = 15)
ax.legend()
plt.tight_layout()
plt.show()

Plotting the linear regression line allows resolution of the trend in annual mean precipitation.

A time step is added to the dataset to visualise the linear regression.

In [ ]:
irish_precip_data['time'] =  np.arange(len(irish_precip_data.index))

On the regression plot, the line of regression indicates that since 1900 the annual mean precipitation is increasing by aprroximately 0.8mm per year. This is a small amount but translates to a 100mm difference over the 123 year period. The gradual increase is so small it is imperceptible, however it is clear that the Irish climate has become wetter over the last 123 years.

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.plot('time', 'annual_mean', data=irish_precip_data, color='0.75')
ax = sns.regplot(x='time', y='annual_mean', data=irish_precip_data, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of Irish Annual Mean Precipitation 1900 - 2022', weight = 'bold', size = 15)

An insight into the monthly variation on precipitation is gained by plotting boxplots of historical monthly data. The boxplots demonstrate that winter and autumn tend to be wetter than spring and summer.

In [ ]:
# create an empty plot
fig, ax = plt.subplots(figsize=(15, 10))

# plot the boxplots
sns.boxplot(x='month', y='median_monthly_precip_mm', 
            data =irish_monthly_precip_data).set_title('Monthly Mean Precipitation 1711-2016', weight='bold', size=15)

The trend in seasonal precipitation gives an indication if changes are more pronounced in one season over another or if the change is spread evenly across all seasons.

To plot seasonal trend subsets of seasonal precipitation data are assigned.

In [ ]:
winter = irish_monthly_precip_data[irish_monthly_precip_data.index.month_name().isin(['November', 'December', 'January'])].sort_index(ascending=True)
spring = irish_monthly_precip_data[irish_monthly_precip_data.index.month_name().isin(['February', 'March', 'April'])].sort_index(ascending=True)
summer = irish_monthly_precip_data[irish_monthly_precip_data.index.month_name().isin(['May', 'June', 'July'])].sort_index(ascending=True)
autumn = irish_monthly_precip_data[irish_monthly_precip_data.index.month_name().isin(['August', 'September', 'October'])].sort_index(ascending=True)

The seasonal trends are plotted with smoothing applied to the seasonal series to make the signal more readable. The trend clearly demonstrates from approximately halfway through the time period that is included in the data, winter months have become wetter in Ireland as highlighted by the gradual increase in the slope of the blue line below.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

ax.plot(moving_avarage_smoothing(winter['median_monthly_precip_mm'], 250), label = 'Winter Median Precipitation', color = 'blue')
ax.plot(moving_avarage_smoothing(spring['median_monthly_precip_mm'], 250), label = 'Spring Median Precipitation', color = 'black')
ax.plot(moving_avarage_smoothing(summer['median_monthly_precip_mm'], 250), label = 'Summer Median Precipitation', color = 'red')
ax.plot(moving_avarage_smoothing(autumn['median_monthly_precip_mm'], 250), label = 'Autumn Median Precipitation', color = 'grey')
ax.set_xlabel("No of Months")
ax.set_ylabel('mm')
ax.legend()
plt.suptitle('Season Months Median Precipitation Ireland 1711 - 2016', weight='bold', size=15)
plt.tight_layout()

This can be compared with local preciptation from Dublin airport, albeit during a more condensed time frame. It is also worth noting that that national data is using the median monthly precipitation value while local data (from Dublin airport) uses the amount of precipitation.

A time step is added to the monthly precipitation data from Dublin airport.

In [ ]:
da_monthly_weather_data['time'] =  np.arange(len(da_monthly_weather_data))

The monthly trend and regression line are plotted. The regression line demonstrates a very shallow slope that indicates that during the period bertween 1941 and 2023 there has been a very small increase in precipitation.

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))

ax.plot('time', 'rain', data=da_monthly_weather_data, color='0.75')
ax = sns.regplot(x='time', y='rain', data=da_monthly_weather_data, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Dublin Airport Monthly Rainfall Data 1941 - 2023', weight = 'bold', size = 15)

Seasonal subsets of Dublin Airport precipitation data are created. This allows a plot of the variation in seasonal months precipitation that can be compared to national seasonal variation in precipitation.

In [ ]:
winter = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['November', 'December', 'January'])].sort_index(ascending=True)
spring = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['February', 'March', 'April'])].sort_index(ascending=True)
summer = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['May', 'June', 'July'])].sort_index(ascending=True)
autumn = da_monthly_weather_data[da_monthly_weather_data.index.month_name().isin(['August', 'September', 'October'])].sort_index(ascending=True)

No discernible trend is obvious in seasonal variation from Dublin Airport which supports the hypothesis that in ths location there has been little to no increase annual precipitation since 1941, in this location.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

ax.plot(moving_avarage_smoothing(winter['rain'], 250), label = 'Dublin Airport Winter Precipitation', color = 'blue')
ax.plot(moving_avarage_smoothing(spring['rain'], 250), label = 'Dublin Airport Spring Precipitation', color = 'black')
ax.plot(moving_avarage_smoothing(summer['rain'], 250), label = 'Dublin Airport Summer Precipitation', color = 'red')
ax.plot(moving_avarage_smoothing(autumn['rain'], 250), label = 'Dublin Airport Autumn Precipitation', color = 'grey')
ax.set_xlabel("No of Months")
ax.set_ylabel('mm of precipitation')
ax.legend()
plt.suptitle('Dublin Airport Season Months Median Precipitation Ireland 1941 - 2023', weight='bold', size=15)
plt.tight_layout()

---

## Predictive Model <a id="predictive-model"></a>

Wang et al. (2023) forecasted temperature anomaly outcomes for 2050 using AutoRegressive Integrated Moving Average (ARIMA 1, 1, 3) models, based on four different time periods and observations. The longest period studied was from 1850 to 2021. Based on this data a global temperature anomaly of 0.92 was forecast in 2050.

Using global temperature anomaly data for the same period a linear regression model can be created and used to predict the temperature in 2050.

In [ ]:
# create a copy of the full global temperature dataset
full_temp_data_training = full_temp_data.copy()

Using iloc the period from 1850 to 2021 is identified.

In [ ]:
full_temp_data_training = full_temp_data_training.iloc[1:173]

In [ ]:
full_temp_data_training

The data is sorted in ascending order based on the year.

In [ ]:
full_temp_data_training = full_temp_data_training.sort_values('year', ascending=True)

A time series is created using the *year* variable.

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html [Accessed 21 Dec. 2023]
full_temp_data_training['time_series'] = pd.to_datetime(full_temp_data_training['year'], format='%Y')

# set time_series column as index
# https://stackoverflow.com/questions/27032052/how-do-i-properly-set-the-datetimeindex-for-a-pandas-datetime-object-in-a-datafr [Accessed 21 Dec. 2023].
full_temp_data_training = full_temp_data_training.set_index('time_series')

A time dummy, which counts off time steps in the series from beginning to end, is added to the data frame. This is used to plot the linear regression line.

In [ ]:
full_temp_data_training['time'] =  np.arange(len(full_temp_data_training.index))

Using Seaborn's *regplot* the linear regression line is plotted over the actual datapoints.

In [ ]:
# an empty plot is created
fig, ax = plt.subplots(figsize=(15,10))

# the linear regression line is plotted over the actual datapoints
ax.plot('time', 'temp_anomaly', data=full_temp_data_training, color='0.75')
ax = sns.regplot(x='time', y='temp_anomaly', data=full_temp_data_training, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of Global Temperature Anomaly 1850 - 2021', weight = 'bold', size = 15)

A date range time series is created for a synthesised dataset that covers the period from 1850 to 2050. This will be fed into the linear regression model to predict the temperature anomaly in 2050.

In [ ]:
# adapted from - https://pandas.pydata.org/docs/reference/api/pandas.date_range.html [Accessed 21 Dec. 2023].
dti = pd.date_range(start="1850", end='2050', freq = 'Y')

The time series is assigned an the index in a Pandas DataFrame.

In [ ]:
# Code adapted from - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html [Accessed 13 Nov. 2023].
synthesised_dataset = pd.DataFrame(index=dti)

In [ ]:
synthesised_dataset['time'] =  np.arange(len(synthesised_dataset.index))

In [ ]:
# code adapted from - https://www.kaggle.com/code/ryanholbrook/linear-regression-with-time-series [Accessed 21 Dec. 2023].

# Training data
X = full_temp_data_training.loc[:, ['time']]  # features
y = full_temp_data_training.loc[:, 'temp_anomaly']  # target

# Train the model
model = LinearRegression()
model.fit(X, y)

In [ ]:
X = synthesised_dataset.loc[:, ['time']]  # features

# Store the fitted values as a time series with the same time index as the synthesised data
y_pred = pd.Series(model.predict(X), index=X.index)

Printing the resultant series indicates that the global temperature anomaly goes from -0.526775 in 1850 to 0.687790 on the 31st of December 2049. Assuming that other variables (CO2 concentrations, methane concentrations, etc.) that are causing the temperature anomaly to change remain as they are now, this equates to $\approx$ 1.2 degrees celsius increase, from 1850, in the global temperature anomaly by 2050.

A predicted global temperature anomaly of 0.68 degrees celsius in 2050 is consistent the the prediction (0.92 degrees celsius) of Wang et al. (2023).

In [ ]:
y_pred

In [ ]:
fig, ax = plt.subplots()

ax.plot(y_pred)
plt.show()

---

## References <a id="references"></a>

admin (2020). Time-Series forecasting with Stochastic Signal Analysis techniques. [online] ML Fundamentals. Available at: https://ataspinar.com/2020/12/22/time-series-forecasting-with-stochastic-signal-analysis-techniques/ [Accessed 28 Dec. 2023].

Annan, J.D. and Hargreaves, J.C. (2013). A new global reconstruction of temperature changes at the Last Glacial Maximum. Climate of the Past, 9(1), pp.367–376. doi:https://doi.org/10.5194/cp-9-367-2013.

Bazin, L., Amaëlle Landais, Bénédicte Lemieux‐Dudon, Toye, H., Veres, D., Frédéric Parrenin, Martinerie, P., Ritz, C., Capron, É., Vladimir Ya. Lipenkov, Marie-France Loutre, Raynaud, D., Vinther, B.M., Svensson, A., Sune Olander Rasmussen, Severi, M., Blunier, T., Leuenberger, M., Fischer, H. and Valérie Masson‐Delmotte (2013). The Antarctic ice core chronology (AICC2012). doi:https://doi.org/10.1594/pangaea.824894.

Bereiter et al. (2014), Revision of the EPICA Dome C CO2 record from 800 to 600 kyr before present, Geophysical Research Letters, doi: 10.1002/2014GL061957.

Bintanja, R., van de Wal, R.S.W. and Oerlemans, J. (2005). Modelled atmospheric temperatures and global sea levels over the past million years. Nature, 437(7055), pp.125–128. doi:https://doi.org/10.1038/nature03975.

climateknowledgeportal.worldbank.org. (n.d.). World Bank Climate Change Knowledge Portal. [online] Available at: https://climateknowledgeportal.worldbank.org/country/ireland/climate-data-historical. [Accessed 20 Dec. 2023].

Dash, S. (2020). Smoothing Techniques for time series data. [online] Medium. Available at: https://medium.com/@srv96/smoothing-techniques-for-time-series-data-91cccfd008a2.[Accessed 21 Dec. 2023].

Datopian (n.d.). Global Temperature Time Series. [online] DataHub. Available at: https://datahub.io/core/global-temp#readme. [Accessed 28 Dec. 2023].

Etheridge, D. M., Steele, L. P., Francey, R. J., and Langenfelds, R. L., 1998, Atmospheric methane between 1000 A.D. and present: Evidence of anthropogenic emissions and climatic variability J. Geophys. Res. Vol. 103, No. D13, p. 15,979 (98JD00923)

cs95 (2018). Pandas Merging 101. [online] Stack Overflow. Available at: https://stackoverflow.com/questions/53645882/pandas-merging-101. [Accessed 18 Dec. 2023].

GeeksforGeeks. (2018). Python | Pandas DataFrame.astype(). [online] Available at: https://www.geeksforgeeks.org/python-pandas-dataframe-astype/. [Accessed 15 Dec. 2023].

GeeksforGeeks. (2020). Change the line opacity in Matplotlib. [online] Available at: https://www.geeksforgeeks.org/change-the-line-opacity-in-matplotlib/. [Accessed 19 Dec. 2023]

Harmel RD, Richardson CW, Hanson CL, Johnson GL (2002) Evaluating the adequacy of simulating maximum and minimum daily air temperature with the normal distribution. J Appl Meteorol 41(7):744–753. https://doi.org/10.1175/1520-0450(2002)041%3c0744:ETAOSM%3e2.0.CO;2

Jouzel, Jean; Masson-Delmotte, Valerie (2007): EPICA Dome C Ice Core 800KYr deuterium data and temperature estimates. PANGAEA, https://doi.org/10.1594/PANGAEA.683655

kaggle.com. (n.d.). Linear Regression With Time Series. [online] Available at: https://www.kaggle.com/code/ryanholbrook/linear-regression-with-time-series. [Accessed 21 Dec. 2023].

Lemieux-Dudon, B., Blayo, E., Petit, J.-R., Waelbroeck, C., Svensson, A., Ritz, C., Barnola, J.-M., Narcisi, B.M. and Parrenin, F. (2010). Consistent dating for Antarctic and Greenland ice cores. Quaternary Science Reviews, 29(1-2), pp.8–20. doi:https://doi.org/10.1016/j.quascirev.2009.11.010.

Lisiecki, L.E. and Raymo, M.E. (2005). A Pliocene-Pleistocene stack of 57 globally distributed benthic δ18O records. Paleoceanography, 20(1), p.n/a-n/a. doi:https://doi.org/10.1029/2004pa001071.

Loulergue, L., Schilt, A., Spahni, R., Masson-Delmotte, V., Blunier, T., Lemieux, B., Barnola, J.-M., Raynaud, D., Stocker, T.F. and Chappellaz, J. (2008). Orbital and millennial-scale features of atmospheric CH4 over the past 800,000 years. Nature, [online] 453(7193), pp.383–386. doi:https://doi.org/10.1038/nature06950.

Lüthi, D., Le Floch, M., Bereiter, B., Blunier, T., Barnola, J.-M., Siegenthaler, U., Raynaud, D., Jouzel, J., Fischer, H., Kawamura, K. and Stocker, T.F. (2008). High-resolution carbon dioxide concentration record 650,000–800,000 years before present. Nature, 453(7193), pp.379–382. doi:https://doi.org/10.1038/nature06949.

Marcott, S.A., Shakun, J.D., Clark, P.U. and Mix, A.C. (2013). A Reconstruction of Regional and Global Temperature for the Past 11,300 Years. Science, 339(6124), pp.1198–1201. doi:https://doi.org/10.1126/science.1228026.

Name, Y. (2022). Pandas filter dates by month, hour, day and last N days & weeks. [online] kanoki. Available at: https://kanoki.org/2022/07/16/pandas-filter-dates-by-month-hour-day-or-last-n-days-weeks/ [Accessed 9 Nov. 2023].

Naveen (2022). How to Rename Columns With List in Pandas. [online] Spark By {Examples}. Available at: https://sparkbyexamples.com/pandas/rename-columns-with-list-in-pandas-dataframe/ [Accessed 13 Dec. 2023].

note.nkmk.me. (2023). pandas: Interpolate NaN (missing values) with interpolate() | note.nkmk.me. [online] Available at: https://note.nkmk.me/en/python-pandas-interpolate/ [Accessed 23 Dec. 2023].

pandas.pydata.org. (2023). pandas.DataFrame — pandas 1.2.4 documentation. [online] Available at: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html. [ Accessed 21 Dec. 2023].

pandas.pydata.org. (n.d.). pandas.concat — pandas 1.3.4 documentation. [online] Available at: https://pandas.pydata.org/docs/reference/api/pandas.concat.html. [Accessed 13 Dec. 2023].

pandas.pydata.org. (n.d.). pandas.DataFrame.drop — pandas 1.2.4 documentation. [online] Available at: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html. [Accessed 13 Dec. 2023].

pandas.pydata.org. (n.d.). pandas.date_range — pandas 2.1.4 documentation. [online] Available at: https://pandas.pydata.org/docs/reference/api/pandas.date_range.html [Accessed 21 Dec. 2023].

pandas.pydata.org. (n.d.). pandas.to_datetime — pandas 1.3.4 documentation. [online] Available at: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html. [Accessed 21 Dec. 2023].

Parrenin, F., Barnola, J.-M. ., Beer, J., Blunier, T., Castellano, E., Chappellaz, J., Dreyfus, G., Fischer, H., Fujita, S., Jouzel, J., Kawamura, K., Lemieux-Dudon, B., Loulergue, L., Masson-Delmotte, V., Narcisi, B., Petit, J.-R. ., Raisbeck, G., Raynaud, D., Ruth, U. and Schwander, J. (2007). The EDC3 chronology for the EPICA Dome C ice core. Climate of the Past, 3(3), pp.485–497. doi:https://doi.org/10.5194/cp-3-485-2007.

practicaldatascience.co.uk. (2022). How to reorder Pandas dataframe columns. [online] Available at: https://practicaldatascience.co.uk/data-science/how-to-reorder-pandas-dataframe-columns. [Accessed 13 Dec. 2023].

saturncloud.io. (2023). How to Sort Pandas DataFrame by One or Multiple Column | Saturn Cloud Blog. [online] Available at: https://saturncloud.io/blog/
how-to-sort-pandas-dataframe-from-one-column/ [Accessed 13 Dec. 2023].

Shakun, J.D., Clark, P.U., He, F., Marcott, S.A., Mix, A.C., Liu, Z., Otto-Bliesner, B., Schmittner, A. and Bard, E. (2012). Global warming preceded by increasing carbon dioxide concentrations during the last deglaciation. Nature, [online] 484(7392), pp.49–54. doi:https://doi.org/10.1038/nature10915.

Snyder, C.W. (2016). Evolution of global temperature over the past two million years. Nature, [online] 538(7624), pp.226–228. doi:https://doi.org/10.1038/nature19798.

Stack Overflow. (n.d.). Cleanly combine year and month columns to single date column with pandas. [online] Available at: https://stackoverflow.com/questions/48304927/cleanly-combine-year-and-month-columns-to-single-date-column-with-pandas [Accessed 21 Dec. 2023].

Stack Overflow. (n.d.). How do I get a list of all the duplicate items using pandas in python? [online] Available at: https://stackoverflow.com/questions/14657241/how-do-i-get-a-list-of-all-the-duplicate-items-using-pandas-in-python [Accessed 22 Dec. 2023].
‌
Stack Overflow. (n.d.). How do I properly set the Datetimeindex for a Pandas datetime object in a dataframe? [online] Available at: https://stackoverflow.com/questions/27032052/how-do-i-properly-set-the-datetimeindex-for-a-pandas-datetime-object-in-a-datafr. [Accessed 21 Dec. 2023].

Stack Overflow. (n.d.). How to find the lag between two time series using cross-correlation. [online] Available at: https://stackoverflow.com/questions/69117617/how-to-find-the-lag-between-two-time-series-using-cross-correlation [Accessed 21 Dec. 2023].

Stack Overflow. (n.d.). Reading tab-delimited file with Pandas - works on Windows, but not on Mac. [online] Available at: https://stackoverflow.com/questions/27896214/reading-tab-delimited-file-with-pandas-works-on-windows-but-not-on-mac [Accessed 15 Dec. 2023].

Stack Overflow. (n.d.). Rounding down values in Pandas dataframe column with NaNs. [online] Available at: https://stackoverflow.com/questions/35873927/rounding-down-values-in-pandas-dataframe-column-with-nans [Accessed 23 Dec. 2023].

Stack Overflow. (n.d.). Secondary axis with twinx(): how to add to legend. [online] Available at: https://stackoverflow.com/questions/5484922/secondary-axis-with-twinx-how-to-add-to-legend. [Accessed 21 Dec. 2023]

US EPA (2022). Importance of Methane. [online] US EPA. Available at: https://www.epa.gov/gmi/importance-methane. [Accessed 18 Dec. 2023]

Veres, D., Bazin, L., Landais, A., Toyé Mahamadou Kele, H., Lemieux-Dudon, B., Parrenin, F., Martinerie, P., Blayo, E., Blunier, T., Capron, E., Chappellaz, J., Rasmussen, S.O., Severi, M., Svensson, A., Vinther, B. and Wolff, E.W. (2013). The Antarctic ice core chronology (AICC2012): an optimized multi-parameter and multi-site dating approach for the last 120 thousand years. Climate of the Past, 9(4), pp.1733–1748. doi:https://doi.org/10.5194/cp-9-1733-2013.

Wang, L., Wang, L., Li, Y. and Wang, J. (2023). A century-long analysis of global warming and earth temperature using a random walk with drift approach. Decision Analytics Journal, [online] 7, p.100237. doi:https://doi.org/10.1016/j.dajour.2023.100237.

Wikipedia Contributors (2023). Irish calendar. [online] Wikipedia. Available at: https://en.wikipedia.org/wiki/Irish_calendar [Accessed 9 Nov. 2023].

www.digitalocean.com. (n.d.). Pandas read_excel() - Reading Excel File in Python | DigitalOcean. [online] Available at: https://www.digitalocean.com/community/tutorials/pandas-read_excel-reading-excel-file-in-python. [19 Dec. 2023].

www.ncei.noaa.gov. (n.d.). Global Surface Temperature Anomalies | National Centers for Environmental Information (NCEI). [online] Available at: https://www.ncei.noaa.gov/access/monitoring/global-temperature-anomalies/.

www.shanelynn.ie. (n.d.). iloc, loc, and ix for data selection in Python Pandas | Shane Lynn. [online] Available at: https://www.shanelynn.ie/pandas-iloc-loc-select-rows-and-columns-dataframe/. [Accessed 19 Dec. 2023].

---

## Associated Reading <a id="associated-reading"></a>

Matplotlib (2012). Matplotlib: Python plotting — Matplotlib 3.1.1 documentation. [online] Matplotlib.org. Available at: https://matplotlib.org/. [Accessed 13 Dec. 2023].

Pandas (2018). Python Data Analysis Library — pandas: Python Data Analysis Library. [online] Pydata.org. Available at: https://pandas.pydata.org/. [Accessed 13 Dec. 2023].

---

Notebook Ends